In [2]:
import os
import numpy as np
import pandas as pd
import datetime as dt

In [1]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 예보데이터 년도별로 합치기

In [ ]:
path = "/content/drive/MyDrive/날씨 빅콘/data"
os.chdir(path) 

In [ ]:
region_list = os.listdir(path + '/전처리데이터/예보데이터_new') 

#not_region = ['결측치처리데이터','결측치처리데이터2'] 
#region_list = [ x for x in region_list if x is not not_region ]

print(len(region_list))
region_list

17


['경남',
 '경북',
 '광주',
 '대전',
 '대구',
 '부산',
 '세종',
 '서울',
 '울산',
 '인천',
 '전북',
 '전남',
 '제주',
 '충북',
 '충남',
 '강원',
 '경기']

In [ ]:
######################### 데이터 다 다운받은 다음 다시 돌리기 #########################
###############################################################################
region_list = os.listdir(path + '/전처리데이터/예보데이터_new') 


for region in region_list:   

  # 파일 경로 
  region_path = path + '/전처리데이터/예보데이터_new/' + region
  print('지역명: ', region) 

  file_list2 = os.listdir(region_path) 

  dong_list = list(set( [i.split('_')[1] for i in file_list2] ))
  print('읍면동: ', dong_list)
  print('읍면동 개수: ', len(dong_list))
  
  print()

지역명:  경남
읍면동:  ['정량동', '동면', '산청읍', '함양읍', '이동면', '부원동', '가포동', '거창읍', '의령읍', '합천읍', '평거동', '장평동', '명곡동', '내이동']
읍면동 개수:  14

지역명:  경북
읍면동:  ['울릉읍', '선주원남동', '청송읍', '풍기읍', '춘양면', '점촌4동', '의성읍', '울진읍', '송도동', '영해면', '동부동', '평화동', '독도', '남원동', '황성동']
읍면동 개수:  15

지역명:  광주
읍면동:  ['운암1동']
읍면동 개수:  1

지역명:  대전
읍면동:  ['온천2동']
읍면동 개수:  1

지역명:  대구
읍면동:  ['효목1동']
읍면동 개수:  1

지역명:  부산
읍면동:  ['대청동']
읍면동 개수:  1

지역명:  세종
읍면동:  ['새롬동']
읍면동 개수:  1

지역명:  서울
읍면동:  ['교남동']
읍면동 개수:  1

지역명:  울산
읍면동:  ['병영2동']
읍면동 개수:  1

지역명:  인천
읍면동:  ['백령면', '동인천동', '불은면', '덕적면']
읍면동 개수:  4

지역명:  전북
읍면동:  ['구암동', '장수읍', '대산면', '덕진동', '상교동', '순창읍', '임실읍', '행안면', '고창읍', '도통동']
읍면동 개수:  10

지역명:  전남
읍면동:  ['

In [ ]:
from datetime import datetime
import pandas

def date_range(start, end):
    start = datetime.strptime(start, "%Y-%m-%d")
    end = datetime.strptime(end, "%Y-%m-%d")
    dates = [date.strftime("%Y-%m-%d") for date in pandas.date_range(start, periods=(end-start).days+1)]
    return dates

In [ ]:
## 모든 지역 / 예보데이터 전처리 

years = np.arange(2012,2017)  # 2012, .., 2016 

for region in region_list: # 결측치처리데이터, 결측치처리데이터2 폴더 제외  

  # 파일 경로 
  region_path = path + '/전처리데이터/예보데이터_new/' + region
  print('지역명: ', region) 

  file_list2 = os.listdir(region_path) 

  dong_list = list(set( [i.split('_')[1] for i in file_list2] ))
  print('읍면동: ', dong_list)
  print('읍면동 개수: ', len(dong_list))
  
  print('=======================================================================')


  for dong in dong_list: 
      df = pd.DataFrame() 

      # csv 파일들을 DataFrame으로 불러와서 concat
      for year in years:  # 연도별 데이터에 대해 전처리 실행 

          data = pd.read_csv(path + '/전처리데이터/예보데이터_new/' +
                            region +'/' + 
                            region +'_'+ dong +'_'+ str(year) +'.csv')  # ex) 경남_동면_2011.csv 
          print(year, ": ", data.shape)

          
          df = pd.concat([df, data]) 

      data_allyears = df.reset_index(drop = True) 
      print("all years data shape: ", data_allyears.shape)  

      # 날짜형으로 변환 
      data_allyears['date'] = pd.to_datetime(data_allyears['date'], format='%Y-%m-%d') 

      # 날짜 데이터프레임 만들기 
      all_ymd = date_range("2012-01-01", "2016-12-31") 
      all_ymd = pd.to_datetime(all_ymd, format='%Y-%m-%d')
      all_ymd2 = pd.DataFrame({'date':all_ymd})
      print("all years length: ", len(all_ymd))

      data = pd.merge(all_ymd2, data_allyears, how='left', on='date')

      # train / test 나누기
      data_train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
      data_test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)


      # train / test 합치기 
      df1 = pd.DataFrame(data_train)
      df2 = pd.DataFrame(data_test)
      df3 = pd.DataFrame(pd.concat([df1, df2], axis=0, ignore_index=True))
      df3.columns = data.columns[1:] 

      # 중복열 제거
      df3 = data.drop_duplicates(keep='first') 

      df3['area'] = region # 울산
      df3['region'] = dong # 읍면동

      print('=======================================================================')
      print(region, " 최종 데이터 shape: ", df3.shape)
      print("365일*5년 + 2일(2/28) = ", 365*5 + 2)
      print(df3.head())
      print(df3.tail())

      print('=======================================================================')
      print('=======================================================================')

      # csv 로 내보내기 
      df3.to_csv(path + '/전처리데이터/예보데이터_new2/' + region +'_'+ dong + '_결측처리.csv')  # ex) 경남_동면_결측처리.csv 


지역명:  경남
읍면동:  ['정량동', '동면', '산청읍', '함양읍', '이동면', '부원동', '가포동', '거창읍', '의령읍', '합천읍', '평거동', '장평동', '명곡동', '내이동']
읍면동 개수:  14
2012 :  (366, 11)
2013 :  (365, 11)
2014 :  (365, 11)
2015 :  (355, 11)
2016 :  (366, 11)
all years data shape:  (1817, 11)
all years length:  1827
경남  최종 데이터 shape:  (1827, 13)
365일*5년 + 2일(2/28) =  1827
        date  rainfall_24  snow_24  temp_24  temp_48  humid_24  humid_48  \
0 2012-01-01          0.0      0.0   2.0250   2.5000    47.500    56.875   
1 2012-01-02          0.0      0.0   1.4000  -0.0625    51.875    45.625   
2 2012-01-03          0.0      0.0  -0.1875   0.6625    39.375    41.875   
3 2012-01-04          0.0      0.0   0.2875   1.7750    46.875    50.625   
4 2012-01-05          0.0      0.0   0.7875   1.5250    50.000    47.500   

   max_temp_24  min_temp_24  wind_24  wind_48    area     region  
0          7.0         -1.9   45.000  320.625  경남  정량동 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



2012 :  (366, 11)
2013 :  (365, 11)
2014 :  (365, 11)
2015 :  (355, 11)
2016 :  (366, 11)
all years data shape:  (1817, 11)
all years length:  1827
전북  최종 데이터 shape:  (1827, 13)
365일*5년 + 2일(2/28) =  1827
        date  rainfall_24  snow_24  temp_24  temp_48  humid_24  humid_48  \
0 2012-01-01          1.0      5.0  -5.0000  -2.3750    74.875    74.125   
1 2012-01-02          0.0      0.0  -2.7375  -3.6875    77.125    71.500   
2 2012-01-03          1.0      1.0  -3.9000  -4.3750    64.125    68.625   
3 2012-01-04          0.0      0.0  -6.1625  -3.8000    75.875    80.125   
4 2012-01-05          0.0      0.0  -5.2125  -4.8000    74.125    66.875   

   max_temp_24  min_temp_24  wind_24  wind_48    area    region  
0          2.8        -11.4  167.500  189.750  전북  도통동  
1          2.9         -7.0  219.250  315.125  전북  도통동  
2          0.9         -9.2  314.250  258.875  전북  도통동  
3          2.8        -12.0  258.375  215.000  전북  도통동  
4  

# 결측값 처리 - interpolate

In [3]:
path = '/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/'

In [4]:
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.csv')]
# 모든 파일명 리스트 
file_name = list(f.split('.')[0] for f in file_list_py) 

In [5]:
file_name

['경남_정량동_결측처리',
 '경남_동면_결측처리',
 '경남_산청읍_결측처리',
 '경남_함양읍_결측처리',
 '경남_이동면_결측처리',
 '경남_부원동_결측처리',
 '경남_가포동_결측처리',
 '경남_거창읍_결측처리',
 '경남_의령읍_결측처리',
 '경남_합천읍_결측처리',
 '경남_평거동_결측처리',
 '경남_장평동_결측처리',
 '경남_명곡동_결측처리',
 '경남_내이동_결측처리',
 '경북_울릉읍_결측처리',
 '경북_선주원남동_결측처리',
 '경북_청송읍_결측처리',
 '경북_풍기읍_결측처리',
 '경북_춘양면_결측처리',
 '경북_점촌4동_결측처리',
 '경북_의성읍_결측처리',
 '경북_울진읍_결측처리',
 '경북_송도동_결측처리',
 '경북_영해면_결측처리',
 '경북_동부동_결측처리',
 '경북_평화동_결측처리',
 '경북_독도_결측처리',
 '경북_남원동_결측처리',
 '경북_황성동_결측처리',
 '광주_운암1동_결측처리',
 '대전_온천2동_결측처리',
 '대구_효목1동_결측ᄎ

In [6]:
len(file_name)

98

# 경북

## 남원동

In [7]:
df = pd.read_csv(path+"경북_남원동_결측처리.csv",index_col=0) 

data = df.reset_index(drop = True)

data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,-2.6750,-1.1875,52.500,51.875,3.0,-6.9,252.125,237.250,경북,남원동
1,2012-01-02,0.0,0.0,-2.5500,-3.0250,56.875,48.750,2.0,-5.9,294.625,300.500,경북,남원동
2,2012-01-03,0.0,0.0,-3.9000,-3.5375,41.250,42.500,1.0,-7.9,300.375,291.375,경북,남원동
3,2012-01-04,0.0,0.0,-4.7875,-1.5625,49.375,52.500,3.0,-8.9,263.125,272.375,경북,남원동
4,2012-01-05,0.0,0.0,-2.8125,-2.3125,51.250,50.625,3.0,-6.9,292.750,203.125,경북,남원동


In [8]:
rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

In [9]:
rain_snow = rain_snow.fillna(0)

In [10]:
data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

In [11]:
train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

In [12]:
train = train.interpolate()
test = test.interpolate()

In [13]:
all_ymd = pd.date_range("2012-01-01", "2016-12-31")

In [14]:
df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

In [15]:
df3

,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,-2.6750,-1.1875,52.500,51.875,3.0,-6.9,252.125,237.250,경북,남원동
1,-2.5500,-3.0250,56.875,48.750,2.0,-5.9,294.625,300.500,경북,남원동
2,-3.9000,-3.5375,41.250,42.500,1.0,-7.9,300.375,291.375,경북,남원동
3,-4.7875,-1.5625,49.375,52.500,3.0,-8.9,263.125,272.375,경북,남원동
4,-2.8125,-2.3125,51.250,50.625,3.0,-6.9,292.750,203.125,경북,남원동
...,...,...,...,...,...,...,...,...,...,...
1822,-0.3750,-0.2500,51.250,42.500,5.0,-6.0,2.125,3.000,경북,남원동
1823,-0.2500,-0.2500,42.500,44.375,6.0,-6.0,3.375,2.500,경북,남원동
1824,-0.3750,1.6250,44.375,60.625,8.0,-5.0,2.500,2.125,경북,남원동
1825,1.0000,3.0000,60.625,64.375,10.0,-2.0,2.125,1.500,경북,남원동


In [16]:
df_final = pd.concat([df3, rain_snow], axis=1)

In [17]:
df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,-2.6750,-1.1875,52.500,51.875,3.0,-6.9,252.125,237.250,경북,남원동
1,2012-01-02,0.0,0.0,-2.5500,-3.0250,56.875,48.750,2.0,-5.9,294.625,300.500,경북,남원동
2,2012-01-03,0.0,0.0,-3.9000,-3.5375,41.250,42.500,1.0,-7.9,300.375,291.375,경북,남원동
3,2012-01-04,0.0,0.0,-4.7875,-1.5625,49.375,52.500,3.0,-8.9,263.125,272.375,경북,남원동
4,2012-01-05,0.0,0.0,-2.8125,-2.3125,51.250,50.625,3.0,-6.9,292.750,203.125,경북,남원동
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-0.3750,-0.2500,51.250,42.500,5.0,-6.0,2.125,3.000,경북,남원동
1823,2016-12-28,0.0,0.0,-0.2500,-0.2500,42.500,44.375,6.0,-6.0,3.375,2.500,경북,남원동
1824,2016-12-29,0.0,0.0,-0.3750,1.6250,44.375,60.625,8.0,-5.0,2.500,2.125,경북,남원동
1825,2016-12-30,0.0,0.0,1.0000,3.0000,60.625,64.375,10.0,-2.0,2.125,1.500,경북,남원동


In [18]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [19]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/경북/경북_남원동_결측처리.csv',index=False)

## 경북_독도

In [20]:
df = pd.read_csv(path+"경북_독도_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [21]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,30.0,25.0,2.0000,3.3750,77.500,83.750,5.0,0.0,247.5000,202.500,경북,독도
1,2012-01-02,10.0,10.0,2.8750,1.1250,83.125,81.250,4.0,-0.9,167.8750,343.750,경북,독도
2,2012-01-03,10.0,5.0,0.7500,3.0000,83.750,61.875,4.0,-0.9,251.3750,313.125,경북,독도
3,2012-01-04,5.0,5.0,3.0000,2.8750,64.375,71.250,4.0,1.0,292.5000,208.125,경북,독도
4,2012-01-05,0.0,0.0,2.1250,1.7500,73.125,75.625,3.0,1.0,159.6250,63.625,경북,독도
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,2.0375,4.0000,62.625,60.625,7.0,0.0,3.3625,5.250,경북,독도
1823,2016-12-28,0.0,0.0,3.7500,5.2875,61.125,58.750,8.0,2.9,7.2375,3.925,경북,독도
1824,2016-12-29,0.0,0.0,5.3875,7.2375,57.500,59.375,9.0,3.0,3.5000,3.875,경북,독도
1825,2016-12-30,0.0,0.0,7.6000,9.3750,59.375,59.375,12.0,6.9,3.8750,3.750,경북,독도


In [22]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [23]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/경북/경북_독도_결측처리.csv',index=False)

## 경북_동부동

In [24]:
df = pd.read_csv(path+"경북_동부동_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [25]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,-2.0625,-0.9375,54.500,52.625,5.0,-6.9,296.875,286.375,경북,동부동
1,2012-01-02,0.0,0.0,-1.0625,-2.3000,54.000,44.375,4.0,-6.9,292.500,315.000,경북,동부동
2,2012-01-03,0.0,0.0,-3.1625,-2.1875,44.375,35.625,3.0,-6.9,299.250,299.500,경북,동부동
3,2012-01-04,0.0,0.0,-2.5500,-0.5750,41.250,48.125,4.0,-6.9,297.250,295.750,경북,동부동
4,2012-01-05,0.0,0.0,-1.9375,-2.8000,44.375,48.750,5.0,-8.9,299.500,298.250,경북,동부동
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-0.5000,-0.7500,55.625,45.625,6.0,-5.0,2.125,2.625,경북,동부동
1823,2016-12-28,0.0,0.0,-1.1250,-0.5000,51.250,46.250,8.0,-6.0,3.250,2.375,경북,동부동
1824,2016-12-29,0.0,0.0,0.2500,1.3750,43.750,59.375,10.0,-6.0,2.250,1.500,경북,동부동
1825,2016-12-30,0.0,0.0,1.8750,3.0000,61.875,63.125,11.0,-3.0,1.875,1.625,경북,동부동


In [26]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [27]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/경북/경북_동부동_결측처리.csv',index=False)

## 경북_선주원남동

In [28]:
df = pd.read_csv(path+"경북_선주원남동_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [29]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,-2.3125,-0.0875,49.375,58.125,4.0,-6.9,292.125,283.375,경북,선주원남동
1,2012-01-02,0.0,0.0,-1.0625,-1.8000,51.250,46.250,4.0,-4.9,288.250,303.250,경북,선주원남동
2,2012-01-03,0.0,0.0,-1.9250,-2.4250,45.625,44.375,2.0,-6.9,302.875,308.500,경북,선주원남동
3,2012-01-04,0.0,0.0,-3.0625,-1.5625,45.625,51.250,3.0,-7.9,297.375,292.375,경북,선주원남동
4,2012-01-05,0.0,0.0,-1.6875,-2.0625,52.500,49.375,4.0,-6.9,311.625,314.500,경북,선주원남동
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-0.2500,0.3750,53.750,46.875,5.0,-5.0,1.375,3.000,경북,선주원남동
1823,2016-12-28,0.0,0.0,0.0000,-0.2500,46.250,49.375,7.0,-5.0,2.750,1.875,경북,선주원남동
1824,2016-12-29,0.0,0.0,-0.2500,1.5000,49.375,63.125,10.0,-5.0,2.125,1.625,경북,선주원남동
1825,2016-12-30,0.0,0.0,1.5000,3.1250,63.125,65.625,11.0,-1.0,1.250,1.375,경북,선주원남동


In [30]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [31]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/경북/경북_선주원남동_결측처리.csv',index=False)

## 경북_송도동

In [32]:
df = pd.read_csv(path+"경북_송도동_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [33]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,1.5000,1.8875,49.000,50.750,6.0,-2.1,266.8750,258.1250,경북,송도동
1,2012-01-02,0.0,0.0,1.2750,-0.7625,49.625,43.875,6.1,-3.9,263.0000,290.0000,경북,송도동
2,2012-01-03,0.0,0.0,-0.9250,-0.2500,39.750,30.625,4.0,-3.9,247.8750,265.2500,경북,송도동
3,2012-01-04,0.0,0.0,-0.4000,1.2875,35.250,38.750,6.1,-3.9,252.5000,242.1250,경북,송도동
4,2012-01-05,0.0,0.0,0.6625,0.9000,37.750,46.375,5.9,-3.9,260.2500,259.3750,경북,송도동
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,1.9000,1.5250,55.375,43.250,7.0,-1.9,2.0625,3.5375,경북,송도동
1823,2016-12-28,0.0,0.0,1.1875,1.9000,45.000,42.875,9.5,-3.0,3.7375,3.0875,경북,송도동
1824,2016-12-29,0.0,0.0,2.6750,4.3500,42.750,57.500,11.1,-1.3,3.0375,2.8125,경북,송도동
1825,2016-12-30,0.0,0.0,4.3625,6.2250,59.625,58.375,12.9,1.6,3.3250,3.3125,경북,송도동


In [34]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [35]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/경북/경북_송도동_결측처리.csv',index=False)

## 경북_영해면

In [36]:
df = pd.read_csv(path+"경북_영해면_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [37]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,0.3000,0.7875,49.000,45.000,5.0,-2.9,278.8750,274.3750,경북,영해면
1,2012-01-02,0.0,0.0,0.6750,-1.1875,48.750,42.500,5.0,-4.9,279.1250,282.2500,경북,영해면
2,2012-01-03,0.0,0.0,-2.0500,-1.4375,38.125,38.750,4.0,-5.9,282.7500,279.0000,경북,영해면
3,2012-01-04,0.0,0.0,-1.1875,-0.1875,38.125,45.000,4.0,-5.9,315.0000,315.0000,경북,영해면
4,2012-01-05,0.0,0.0,-0.5625,-0.0750,45.625,55.000,4.0,-4.9,287.7500,316.6250,경북,영해면
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,1.1125,0.2375,53.000,39.500,6.1,-2.7,2.9000,3.3625,경북,영해면
1823,2016-12-28,0.0,0.0,0.1375,1.1125,40.625,38.250,9.0,-3.8,3.4875,3.8625,경북,영해면
1824,2016-12-29,0.0,0.0,1.2875,3.3750,37.125,53.125,10.0,-2.9,4.3375,3.4375,경북,영해면
1825,2016-12-30,0.0,0.0,3.3500,5.0000,53.625,55.500,12.0,0.0,3.4750,2.8250,경북,영해면


In [38]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [39]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/경북/경북_영해면_결측처리.csv',index=False)

## 경북_울릉읍

In [40]:
df = pd.read_csv(path+"경북_울릉읍_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [41]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,30.0,25.0,0.0250,2.5000,77.500,83.750,3.0,-1.9,247.500,202.500,경북,울릉읍
1,2012-01-02,10.0,10.0,0.6375,-1.1500,83.125,81.250,3.0,-2.9,216.250,111.125,경북,울릉읍
2,2012-01-03,10.0,10.0,-0.7000,1.8750,86.875,61.250,3.0,-1.9,290.125,304.125,경북,울릉읍
3,2012-01-04,5.0,5.0,1.6250,1.5000,64.375,71.250,2.0,0.0,292.500,208.125,경북,울릉읍
4,2012-01-05,0.0,0.0,0.0375,-0.5625,73.125,75.625,1.0,-1.9,159.625,63.625,경북,울릉읍
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,5.0,5.0,0.6250,1.6250,63.750,60.625,6.0,-1.0,4.000,4.375,경북,울릉읍
1823,2016-12-28,10.0,6.0,2.0000,3.3750,60.000,55.000,7.0,1.0,7.625,3.500,경북,울릉읍
1824,2016-12-29,0.0,0.0,3.1250,5.0000,55.625,58.125,8.0,1.0,3.500,3.625,경북,울릉읍
1825,2016-12-30,0.0,0.0,5.0000,6.8750,59.375,54.375,11.0,4.0,3.625,3.625,경북,울릉읍


In [42]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [43]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/경북/경북_울릉읍_결측처리.csv',index=False)

## 경북_울진읍

In [44]:
df = pd.read_csv(path+"경북_울진읍_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [45]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,5.0,5.0,1.2750,1.4000,60.125,46.000,6.0,-2.9,238.250,280.750,경북,울진읍
1,2012-01-02,0.0,0.0,0.3000,-1.1875,51.875,43.750,6.0,-4.9,283.500,282.000,경북,울진읍
2,2012-01-03,0.0,0.0,-1.1875,-0.3250,42.500,38.750,5.0,-5.9,275.250,274.750,경북,울진읍
3,2012-01-04,0.0,0.0,-0.6875,0.4250,36.875,51.875,5.0,-5.9,315.000,315.000,경북,울진읍
4,2012-01-05,0.0,0.0,0.0500,0.6750,47.500,55.625,5.0,-4.9,313.125,328.125,경북,울진읍
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,1.7500,0.3750,63.750,46.875,6.0,-1.0,3.625,3.875,경북,울진읍
1823,2016-12-28,0.0,0.0,0.5000,2.1250,50.625,43.750,9.0,-3.0,3.750,3.750,경북,울진읍
1824,2016-12-29,0.0,0.0,2.0000,3.8750,47.500,52.500,11.0,-3.0,3.750,3.750,경북,울진읍
1825,2016-12-30,0.0,0.0,3.5000,5.7500,58.125,51.250,13.0,0.0,3.750,3.375,경북,울진읍


In [46]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [47]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/경북/경북_울진읍_결측처리.csv',index=False)

## 경북_의성읍

In [48]:
df = pd.read_csv(path+"경북_의성읍_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [49]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,-4.4375,-1.4375,53.125,58.125,4.0,-10.9,271.500,272.625,경북,의성읍
1,2012-01-02,0.0,0.0,-3.4375,-4.7875,60.000,51.250,4.0,-9.9,286.875,293.125,경북,의성읍
2,2012-01-03,0.0,0.0,-4.6625,-5.4125,51.250,56.250,2.0,-13.9,290.500,281.500,경북,의성읍
3,2012-01-04,0.0,0.0,-7.1625,-5.3125,52.500,56.875,4.0,-13.9,281.250,292.500,경북,의성읍
4,2012-01-05,0.0,0.0,-4.9375,-5.8000,49.375,52.500,3.0,-13.9,277.625,313.375,경북,의성읍
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-2.8750,-2.6250,65.000,55.625,6.0,-10.0,1.125,1.750,경북,의성읍
1823,2016-12-28,0.0,0.0,-2.5000,-2.5000,55.625,60.625,6.0,-10.0,1.875,1.250,경북,의성읍
1824,2016-12-29,0.0,0.0,-2.5000,-0.3750,60.625,74.375,10.0,-9.0,1.250,1.500,경북,의성읍
1825,2016-12-30,0.0,0.0,-0.7500,0.7500,74.375,75.000,12.0,-6.0,1.500,1.375,경북,의성읍


In [50]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [51]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/경북/경북_의성읍_결측처리.csv',index=False)

## 경북_점촌4동

In [52]:
df = pd.read_csv(path+"경북_점촌4동_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [53]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,-3.2625,-1.7625,52.625,51.250,3.1,-7.9,234.750,192.000,경북,점촌4동
1,2012-01-02,0.0,0.0,-3.2500,-3.9375,57.125,48.375,2.1,-7.0,298.625,300.750,경북,점촌4동
2,2012-01-03,0.0,0.0,-4.2500,-3.9000,41.375,42.375,0.0,-8.0,299.625,269.875,경북,점촌4동
3,2012-01-04,0.0,0.0,-5.6125,-2.5250,49.375,52.625,3.2,-9.9,277.750,248.000,경북,점촌4동
4,2012-01-05,0.0,0.0,-3.9000,-3.0500,51.375,50.875,3.1,-10.0,223.375,129.875,경북,점촌4동
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-1.3750,-1.5000,51.875,43.750,5.0,-7.0,2.375,2.500,경북,점촌4동
1823,2016-12-28,0.0,0.0,-1.5000,-1.2500,43.750,47.500,6.0,-7.0,3.000,2.125,경북,점촌4동
1824,2016-12-29,0.0,0.0,-1.2500,0.6250,47.500,62.500,9.0,-6.0,2.125,1.750,경북,점촌4동
1825,2016-12-30,0.0,0.0,0.1250,2.0000,62.500,62.500,10.0,-4.0,1.750,1.625,경북,점촌4동


In [54]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [55]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/경북/경북_점촌4동_결측처리.csv',index=False)

## 경북_청송읍

In [56]:
df = pd.read_csv(path+"경북_청송읍_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [57]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,-5.0250,-2.5250,53.625,58.625,3.1,-12.1,262.750,265.500,경북,청송읍
1,2012-01-02,0.0,0.0,-3.8375,-5.3750,55.000,47.500,3.1,-10.1,292.000,289.875,경북,청송읍
2,2012-01-03,0.0,0.0,-5.9500,-6.4750,51.375,57.375,1.0,-14.1,286.500,278.250,경북,청송읍
3,2012-01-04,0.0,0.0,-8.3125,-6.3375,53.250,57.500,2.1,-15.1,248.750,232.750,경북,청송읍
4,2012-01-05,0.0,0.0,-6.3375,-6.8000,49.250,49.750,3.0,-16.1,274.125,231.875,경북,청송읍
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-3.3750,-3.1250,62.500,46.250,5.0,-9.0,2.500,5.875,경북,청송읍
1823,2016-12-28,0.0,0.0,-3.1250,-2.5000,46.250,48.750,6.0,-10.0,5.625,4.375,경북,청송읍
1824,2016-12-29,0.0,0.0,-2.5000,-0.2500,48.750,63.125,8.0,-8.0,1.375,1.250,경북,청송읍
1825,2016-12-30,0.0,0.0,-1.1250,0.8750,63.125,65.000,10.0,-6.0,1.250,3.750,경북,청송읍


In [58]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [59]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/경북/경북_청송읍_결측처리.csv',index=False)

## 경북_춘양면

In [85]:
df = pd.read_csv(path+"경북_춘양면_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

In [86]:
test.head()

,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
1461,-0.125,0.250,77.500,75.625,10.0,NaN,1.250,1.250,경북,춘양면
1462,1.375,1.625,79.375,75.625,11.0,NaN,1.250,1.375,경북,춘양면
1463,0.750,-2.250,68.125,55.000,8.0,NaN,1.125,1.500,경북,춘양면
1464,-3.375,-3.500,60.000,60.000,4.0,NaN,1.250,1.875,경북,춘양면
1465,-3.750,-4.500,60.000,53.125,4.0,NaN,1.375,1.250,경북,춘양면


In [87]:
test['min_temp_24'][1461] = -10

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [88]:
test.head()

,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
1461,-0.125,0.250,77.500,75.625,10.0,-10.0,1.250,1.250,경북,춘양면
1462,1.375,1.625,79.375,75.625,11.0,NaN,1.250,1.375,경북,춘양면
1463,0.750,-2.250,68.125,55.000,8.0,NaN,1.125,1.500,경북,춘양면
1464,-3.375,-3.500,60.000,60.000,4.0,NaN,1.250,1.875,경북,춘양면
1465,-3.750,-4.500,60.000,53.125,4.0,NaN,1.375,1.250,경북,춘양면


In [89]:
train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [90]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,-4.6750,-3.8000,63.750,58.500,2.0,-10.9,284.875,279.250,경북,춘양면
1,2012-01-02,0.0,0.0,-4.6750,-6.1500,58.750,48.750,3.0,-10.9,281.500,284.125,경북,춘양면
2,2012-01-03,0.0,0.0,-6.4000,-7.0375,48.125,51.250,0.0,-14.9,283.500,281.500,경북,춘양면
3,2012-01-04,0.0,0.0,-9.2750,-6.9125,53.750,60.000,1.0,-14.9,315.000,315.000,경북,춘양면
4,2012-01-05,0.0,0.0,-7.1625,-7.4125,62.500,63.750,1.0,-15.9,290.125,285.875,경북,춘양면
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-3.5000,-3.6250,66.875,48.125,4.0,-10.0,1.750,1.500,경북,춘양면
1823,2016-12-28,0.0,0.0,-3.7500,-3.6250,48.125,58.750,6.0,-10.0,1.875,1.375,경북,춘양면
1824,2016-12-29,0.0,0.0,-3.6250,-1.2500,58.750,72.500,8.0,-9.0,1.375,1.625,경북,춘양면
1825,2016-12-30,0.0,0.0,-1.6250,-0.3750,72.500,70.000,8.0,-7.0,1.625,1.375,경북,춘양면


In [95]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [96]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/경북/경북_춘양면_결측처리.csv',index=False)

## 경북_평화동

In [97]:
df = pd.read_csv(path+"경북_평화동_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [98]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,-2.6875,-0.9500,53.125,58.125,3.0,-7.9,268.000,269.500,경북,평화동
1,2012-01-02,0.0,0.0,-1.9375,-3.6625,56.875,49.375,3.0,-6.9,286.875,299.375,경북,평화동
2,2012-01-03,0.0,0.0,-4.1625,-4.2875,46.875,49.375,1.0,-10.9,289.500,274.875,경북,평화동
3,2012-01-04,0.0,0.0,-5.4125,-3.8000,46.875,51.875,3.0,-10.9,253.125,236.250,경북,평화동
4,2012-01-05,0.0,0.0,-3.5625,-3.3125,47.500,48.125,3.0,-10.9,280.500,207.375,경북,평화동
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-1.2500,-1.2500,62.500,48.125,5.0,-7.0,1.500,2.500,경북,평화동
1823,2016-12-28,0.0,0.0,-1.1250,-1.1250,48.125,53.750,6.0,-7.0,2.750,1.875,경북,평화동
1824,2016-12-29,0.0,0.0,-1.1250,1.3750,53.750,68.125,9.0,-6.0,1.875,1.625,경북,평화동
1825,2016-12-30,0.0,0.0,0.8750,2.3750,68.125,69.375,11.0,-5.0,1.625,1.500,경북,평화동


In [99]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [100]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/경북/경북_평화동_결측처리.csv',index=False)

## 경북_풍기읍

In [101]:
df = pd.read_csv(path+"경북_풍기읍_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [102]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,-3.7875,-2.1625,52.500,51.250,3.0,-8.9,247.500,292.500,경북,풍기읍
1,2012-01-02,0.0,0.0,-2.9125,-4.0250,57.500,48.750,2.0,-6.9,298.125,315.000,경북,풍기읍
2,2012-01-03,0.0,0.0,-5.2750,-4.9000,41.250,42.500,0.0,-9.9,315.000,315.000,경북,풍기읍
3,2012-01-04,0.0,0.0,-5.7750,-3.1875,50.000,51.875,2.0,-11.9,315.000,270.000,경북,풍기읍
4,2012-01-05,0.0,0.0,-4.4125,-4.1625,51.250,50.625,3.0,-9.9,292.500,247.500,경북,풍기읍
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-1.3750,-1.6250,60.000,45.000,4.0,-6.0,2.625,4.000,경북,풍기읍
1823,2016-12-28,0.0,0.0,-1.7500,-2.1250,45.000,51.875,6.0,-8.0,5.000,3.500,경북,풍기읍
1824,2016-12-29,0.0,0.0,-2.1250,0.5000,51.875,64.375,8.0,-7.0,3.500,3.375,경북,풍기읍
1825,2016-12-30,0.0,0.0,0.2500,1.7500,64.375,64.375,8.0,-4.0,3.375,3.000,경북,풍기읍


In [103]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [104]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/경북/경북_풍기읍_결측처리.csv',index=False)

## 경북_황성동

In [105]:
df = pd.read_csv(path+"경북_황성동_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [106]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,-0.4250,0.1500,51.125,50.625,6.0,-5.8,286.6250,273.8750,경북,황성동
1,2012-01-02,0.0,0.0,0.0250,-1.8500,52.875,43.875,5.1,-5.9,280.0000,314.5000,경북,황성동
2,2012-01-03,0.0,0.0,-2.2125,-1.2250,45.750,36.125,3.0,-5.9,309.6250,300.7500,경북,황성동
3,2012-01-04,0.0,0.0,-1.4625,0.3875,41.000,49.625,5.0,-5.9,293.3750,296.8750,경북,황성동
4,2012-01-05,0.0,0.0,-0.8000,-1.2000,38.750,45.125,5.0,-6.8,289.3750,283.0000,경북,황성동
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-0.0250,0.3500,55.125,44.375,6.3,-4.0,2.1750,4.6375,경북,황성동
1823,2016-12-28,0.0,0.0,-0.2000,0.1125,45.625,43.375,9.1,-5.1,4.5875,3.9250,경북,황성동
1824,2016-12-29,0.0,0.0,0.1125,1.9500,41.125,58.750,10.1,-3.8,4.8500,3.2250,경북,황성동
1825,2016-12-30,0.0,0.0,2.5625,3.6250,59.375,59.750,11.1,-1.2,3.8375,3.6125,경북,황성동


In [107]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [108]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/경북/경북_황성동_결측처리.csv',index=False)

# 인천

## 덕적면

In [109]:
df = pd.read_csv(path+"인천_덕적면_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [110]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,5.0,1.0,-0.8000,0.4250,65.375,68.750,1.1,-2.6,317.5000,274.3750,인천,덕적면
1,2012-01-02,0.0,0.0,-1.4250,-2.5875,64.750,54.625,0.2,-3.2,251.1250,319.6250,인천,덕적면
2,2012-01-03,1.0,0.0,-3.0375,-2.9625,62.875,59.000,-1.5,-5.6,321.2500,244.3750,인천,덕적면
3,2012-01-04,0.0,0.0,-2.4000,-1.3000,60.250,62.000,1.1,-5.1,261.0000,291.0000,인천,덕적면
4,2012-01-05,0.0,0.0,-1.7500,-1.3750,67.500,61.750,1.7,-4.3,299.5000,264.5000,인천,덕적면
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-1.3000,-0.7625,59.250,52.625,2.7,-4.7,5.4000,10.4250,인천,덕적면
1823,2016-12-28,1.0,0.0,-1.1625,-0.0375,53.250,65.125,6.7,-3.2,10.0000,4.4500,인천,덕적면
1824,2016-12-29,0.0,0.0,0.4625,4.3125,65.250,71.750,8.6,-2.0,5.1000,3.9250,인천,덕적면
1825,2016-12-30,0.0,0.0,2.9500,5.2500,71.875,72.125,9.6,-1.4,3.9375,4.0500,인천,덕적면


In [111]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [112]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/인천/인천_덕적면_결측처리.csv',index=False)

## 동인천동

In [113]:
df = pd.read_csv(path+"인천_동인천동_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [114]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,5.0,1.0,-2.9125,-1.9250,68.125,75.625,1.0,-5.9,315.000,303.750,인천,동인천동
1,2012-01-02,0.0,0.0,-2.5375,-3.5250,65.750,51.625,0.0,-5.9,282.500,312.000,인천,동인천동
2,2012-01-03,0.0,0.0,-4.6500,-3.2875,53.750,50.625,0.0,-7.9,312.125,284.250,인천,동인천동
3,2012-01-04,0.0,0.0,-4.1500,-2.7875,55.000,52.500,1.0,-7.9,293.875,245.625,인천,동인천동
4,2012-01-05,0.0,0.0,-2.5500,-2.0625,49.375,48.125,1.0,-5.9,315.000,298.875,인천,동인천동
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-2.7500,-2.2500,47.500,50.000,3.0,-7.0,2.375,3.875,인천,동인천동
1823,2016-12-28,1.0,5.0,-2.8750,-1.6250,51.250,54.375,5.0,-6.0,3.875,2.375,인천,동인천동
1824,2016-12-29,0.0,0.0,-1.6250,2.6250,55.000,68.750,8.0,-7.0,2.250,1.875,인천,동인천동
1825,2016-12-30,0.0,0.0,2.5000,4.2500,71.250,76.250,8.0,-1.0,2.000,2.125,인천,동인천동


In [115]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [116]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/인천/인천_동인천동_결측처리.csv',index=False)

## 백령면

In [117]:
df = pd.read_csv(path+"인천_백령면_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [118]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,1.0,1.0,-2.7500,-1.5125,73.375,83.125,1.0,-5.2,315.5000,300.1250,인천,백령면
1,2012-01-02,0.0,0.0,-1.9125,-2.7375,72.875,67.875,0.2,-4.1,274.7500,329.8750,인천,백령면
2,2012-01-03,0.0,0.0,-3.5125,-1.3625,70.250,69.375,0.8,-5.1,332.1250,307.6250,인천,백령면
3,2012-01-04,0.0,0.0,-2.2750,-1.3625,69.875,62.875,1.0,-5.1,309.8750,301.5000,인천,백령면
4,2012-01-05,0.0,0.0,-1.2750,-0.9000,72.375,69.625,2.0,-3.1,315.7500,299.3750,인천,백령면
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-0.9625,-2.0000,61.750,56.250,3.9,-4.2,5.2875,7.2250,인천,백령면
1823,2016-12-28,0.0,0.0,-2.1375,1.5875,56.250,56.875,5.9,-3.2,7.2125,2.8875,인천,백령면
1824,2016-12-29,0.0,0.0,-0.0750,2.5375,61.875,66.250,7.0,-3.4,2.7250,2.7875,인천,백령면
1825,2016-12-30,0.0,0.0,2.4375,4.4625,66.125,68.875,7.0,-0.5,3.3750,4.4625,인천,백령면


In [119]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [120]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/인천/인천_백령면_결측처리.csv',index=False)

## 불은면

In [121]:
df = pd.read_csv(path+"인천_불은면_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [122]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,1.0,1.0,-4.9375,-3.1625,68.375,74.875,1.2,-9.7,315.2500,304.125,인천,불은면
1,2012-01-02,0.0,0.0,-4.4375,-5.8250,64.375,47.250,-0.8,-8.6,218.7500,311.750,인천,불은면
2,2012-01-03,0.0,0.0,-6.4125,-5.2500,54.875,53.125,-0.7,-10.7,312.6250,255.250,인천,불은면
3,2012-01-04,0.0,0.0,-5.7625,-3.8125,55.625,53.125,1.1,-11.5,256.5000,217.000,인천,불은면
4,2012-01-05,0.0,0.0,-4.8000,-3.8125,58.250,60.750,1.2,-9.6,314.0000,298.875,인천,불은면
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-4.0500,-3.3625,54.250,47.750,3.1,-8.9,1.6250,4.000,인천,불은면
1823,2016-12-28,1.0,1.0,-3.5750,-2.6125,47.500,62.000,5.7,-8.8,3.9875,1.375,인천,불은면
1824,2016-12-29,0.0,0.0,-2.7750,1.0875,60.625,73.000,7.2,-8.7,1.4250,1.525,인천,불은면
1825,2016-12-30,0.0,0.0,0.9875,2.5625,72.875,72.250,8.1,-2.7,1.5625,1.400,인천,불은면


In [123]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [124]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/인천/인천_불은면_결측처리.csv',index=False)

# 세종

## 새롬동

In [125]:
df = pd.read_csv(path+"세종_새롬동_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [126]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,-3.4500,-1.5125,64.875,69.250,1.9,-9.7,236.125,234.2500,세종,새롬동
1,2012-01-02,0.0,0.0,-2.1500,-3.3000,67.875,56.500,1.7,-5.9,231.625,308.3750,세종,새롬동
2,2012-01-03,0.0,0.0,-4.4000,-3.3625,65.250,64.875,0.7,-9.1,309.500,258.8750,세종,새롬동
3,2012-01-04,0.0,0.0,-4.0625,-2.5750,64.250,63.500,2.8,-8.9,185.875,253.1250,세종,새롬동
4,2012-01-05,0.0,0.0,-3.6625,-3.6625,58.250,55.625,2.9,-9.5,299.000,203.7500,세종,새롬동
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-4.5250,-2.6875,62.875,64.125,4.1,-11.5,0.575,1.8375,세종,새롬동
1823,2016-12-28,1.0,1.0,-2.6875,-3.9125,65.250,68.500,5.8,-10.6,1.800,0.7750,세종,새롬동
1824,2016-12-29,0.0,0.0,-2.7750,0.6000,64.875,82.625,9.1,-8.3,0.800,0.5500,세종,새롬동
1825,2016-12-30,0.0,0.0,0.2750,1.9375,83.625,81.500,10.1,-2.9,0.575,0.5000,세종,새롬동


In [127]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [128]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/세종/세종_새롬동_결측처리.csv',index=False)

#전남

## 강진읍

In [129]:
df = pd.read_csv(path+"전남_강진읍_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [130]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,1.0375,1.6500,58.125,66.250,5.0,5.0,329.375,297.875,전남,강진읍
1,2012-01-02,0.0,0.0,0.3000,-0.3125,68.125,58.750,5.0,2.0,318.750,324.750,전남,강진읍
2,2012-01-03,1.0,1.0,-0.4375,0.0500,60.000,61.250,4.0,2.0,325.750,333.625,전남,강진읍
3,2012-01-04,0.0,0.0,-0.4375,0.5500,63.750,63.125,5.0,4.0,277.375,278.375,전남,강진읍
4,2012-01-05,0.0,0.0,0.1750,0.4250,63.125,49.375,6.0,5.0,341.500,346.125,전남,강진읍
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,0.0000,0.6250,50.000,63.750,6.0,-4.0,2.750,2.125,전남,강진읍
1823,2016-12-28,0.0,0.0,0.7500,0.2500,62.500,63.750,8.0,-3.0,2.125,2.125,전남,강진읍
1824,2016-12-29,0.0,0.0,0.0000,1.5000,63.125,70.000,11.0,-3.0,1.500,1.500,전남,강진읍
1825,2016-12-30,0.0,0.0,1.6250,4.0000,70.000,70.000,11.0,-3.0,1.500,1.500,전남,강진읍


In [131]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [132]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/전남/전남_강진읍_결측처리.csv',index=False)

## 고흥읍

In [133]:
df = pd.read_csv(path+"전남_고흥읍_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [134]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,0.9500,1.8250,60.625,67.500,6.8,-2.9,289.8750,293.7500,전남,고흥읍
1,2012-01-02,0.0,0.0,0.1750,-0.5875,59.500,52.000,5.0,-3.7,315.6250,307.3750,전남,고흥읍
2,2012-01-03,0.0,0.0,-0.4625,-0.1375,52.625,56.875,5.0,-3.9,313.5000,314.7500,전남,고흥읍
3,2012-01-04,0.0,0.0,-0.8625,0.5875,58.625,61.125,6.8,-4.8,318.3750,318.8750,전남,고흥읍
4,2012-01-05,0.0,0.0,0.1375,0.2875,58.875,49.375,7.9,-5.7,318.6250,204.2500,전남,고흥읍
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,0.3000,1.1250,51.625,55.250,7.9,-4.8,2.0375,2.9750,전남,고흥읍
1823,2016-12-28,0.0,0.0,0.8875,1.0750,55.875,53.375,8.8,-3.6,3.2750,1.9875,전남,고흥읍
1824,2016-12-29,0.0,0.0,0.6625,3.4875,53.000,64.375,10.9,-3.8,1.9500,1.7250,전남,고흥읍
1825,2016-12-30,0.0,0.0,3.1625,4.8125,66.375,65.250,11.9,-2.5,1.5125,1.4500,전남,고흥읍


In [135]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [136]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/전남/전남_고흥읍_결측처리.csv',index=False)

## 군서면

In [137]:
df = pd.read_csv(path+"전남_군서면_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [138]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,5.0,5.0,-1.3250,-0.3000,67.875,79.625,3.6,-5.9,269.1250,178.1250,전남,군서면
1,2012-01-02,0.0,0.0,-0.8125,-3.0250,80.625,71.750,3.0,-4.9,244.1250,328.3750,전남,군서면
2,2012-01-03,1.0,5.0,-2.7625,-1.8125,80.125,75.875,1.7,-4.8,303.0000,191.7500,전남,군서면
3,2012-01-04,5.0,5.0,-1.5125,-1.0875,68.500,70.625,3.4,-4.5,277.2500,88.8750,전남,군서면
4,2012-01-05,0.0,0.0,-2.0375,-2.0500,67.250,64.125,3.3,-7.5,255.7500,172.6250,전남,군서면
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-1.2250,-0.4500,63.375,66.250,4.2,-5.6,2.0750,2.8625,전남,군서면
1823,2016-12-28,1.0,1.0,-0.9125,-0.9750,67.250,68.625,7.6,-5.8,2.8875,2.1125,전남,군서면
1824,2016-12-29,0.0,0.0,-1.2125,1.8625,63.250,67.500,9.0,-5.7,1.3750,1.4000,전남,군서면
1825,2016-12-30,0.0,0.0,1.1625,3.5250,65.125,66.375,10.0,-4.4,1.4750,1.1000,전남,군서면


In [139]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [140]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/전남/전남_군서면_결측처리.csv',index=False)

## 군외면

In [141]:
df = pd.read_csv(path+"전남_군외면_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [142]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,1.0,0.0,0.7500,1.8625,58.125,64.625,5.9,-2.1,327.1250,294.0000,전남,군외면
1,2012-01-02,0.0,0.0,0.6875,-0.5875,65.750,60.750,4.8,-2.1,305.0000,319.0000,전남,군외면
2,2012-01-03,1.0,1.0,-0.5250,0.0625,63.375,60.625,3.9,-2.0,318.6250,285.0000,전남,군외면
3,2012-01-04,0.0,0.0,-0.3000,0.8250,63.750,64.500,4.9,-3.0,203.6250,243.8750,전남,군외면
4,2012-01-05,0.0,0.0,0.0500,0.3375,63.625,51.250,5.9,-3.1,344.5000,348.1250,전남,군외면
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,0.5500,1.5250,53.125,60.875,5.9,-1.9,3.0000,5.2375,전남,군외면
1823,2016-12-28,0.0,0.0,1.4000,0.4125,60.250,59.000,8.5,-1.8,4.6375,2.8250,전남,군외면
1824,2016-12-29,0.0,0.0,0.8625,4.2000,57.125,66.500,9.8,-1.1,2.9875,1.9500,전남,군외면
1825,2016-12-30,0.0,0.0,4.5250,5.8125,66.500,65.500,10.8,0.1,1.9500,3.1250,전남,군외면


In [143]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [144]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/전남/전남_군외면_결측처리.csv',index=False)

## 득량면

In [145]:
df = pd.read_csv(path+"전남_득량면_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [146]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,-0.6125,-0.1375,59.500,64.500,4.6,-4.3,286.0000,293.6250,전남,득량면
1,2012-01-02,0.0,0.0,-1.3250,-2.0750,64.625,59.500,3.2,-4.9,320.1250,309.8750,전남,득량면
2,2012-01-03,0.0,0.0,-2.0375,-1.6250,58.500,57.250,3.5,-5.3,316.7500,318.7500,전남,득량면
3,2012-01-04,0.0,0.0,-1.6875,-0.6500,60.125,60.000,5.5,-5.1,321.1250,322.2500,전남,득량면
4,2012-01-05,0.0,0.0,-1.5625,-0.6750,63.375,56.500,5.8,-7.1,315.6250,205.2500,전남,득량면
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-0.7625,-0.2625,51.000,59.625,6.5,-5.7,1.6625,2.7500,전남,득량면
1823,2016-12-28,0.0,0.0,-0.0125,-0.1500,57.125,54.750,8.3,-4.8,3.1000,1.9375,전남,득량면
1824,2016-12-29,0.0,0.0,-0.2250,1.8125,55.000,69.125,10.3,-4.7,2.1500,1.6250,전남,득량면
1825,2016-12-30,0.0,0.0,1.8000,2.8125,70.000,68.125,10.4,-3.6,1.5375,1.4500,전남,득량면


In [147]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [148]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/전남/전남_득량면_결측처리.csv',index=False)

## 승주읍

In [149]:
df = pd.read_csv(path+"전남_승주읍_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [150]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,-0.3750,0.6375,61.750,58.625,4.8,-4.6,249.8750,290.1250,전남,승주읍
1,2012-01-02,0.0,0.0,-0.4625,-1.6000,61.625,58.625,3.7,-3.8,304.8750,318.1250,전남,승주읍
2,2012-01-03,0.0,0.0,-2.4250,-2.1625,54.250,54.750,2.0,-5.8,317.8750,329.6250,전남,승주읍
3,2012-01-04,0.0,0.0,-1.9500,-0.6125,54.875,54.000,4.7,-5.2,208.1250,331.6250,전남,승주읍
4,2012-01-05,0.0,0.0,-1.3375,-0.1125,54.500,43.875,4.7,-6.4,315.7500,330.5000,전남,승주읍
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-0.8625,-0.3250,57.000,60.125,6.0,-6.2,1.9125,2.4375,전남,승주읍
1823,2016-12-28,0.0,0.0,-0.1250,0.2375,58.000,59.875,7.8,-3.8,3.0000,2.0875,전남,승주읍
1824,2016-12-29,0.0,0.0,-0.1500,2.0000,59.000,74.125,10.1,-4.6,1.8250,1.0375,전남,승주읍
1825,2016-12-30,0.0,0.0,2.1375,3.4250,75.000,72.000,11.1,-2.5,1.3875,1.1375,전남,승주읍


In [151]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [152]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/전남/전남_승주읍_결측처리.csv',index=False)

## 연산동

In [153]:
df = pd.read_csv(path+"전남_연산동_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [154]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,5.0,5.0,1.1500,1.3750,70.625,73.750,4.0,-1.9,295.000,308.250,전남,연산동
1,2012-01-02,0.0,0.0,0.9000,-0.3250,73.750,78.125,4.0,-1.9,320.125,332.500,전남,연산동
2,2012-01-03,5.0,5.0,-0.7125,0.3875,80.000,70.000,3.0,-1.9,332.625,339.625,전남,연산동
3,2012-01-04,0.0,0.0,-0.2000,0.7750,71.875,72.500,5.0,-2.9,299.875,337.000,전남,연산동
4,2012-01-05,0.0,0.0,-0.2000,0.0625,76.875,67.500,5.0,-3.9,204.625,57.125,전남,연산동
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,0.2500,2.0000,65.000,78.750,6.0,-3.0,3.375,5.500,전남,연산동
1823,2016-12-28,0.0,0.0,2.1250,0.6250,75.000,68.750,8.0,-2.0,5.500,3.000,전남,연산동
1824,2016-12-29,0.0,0.0,0.3750,3.5000,68.750,67.875,10.0,-2.0,2.500,2.500,전남,연산동
1825,2016-12-30,0.0,0.0,3.5000,4.8750,67.500,67.500,11.0,-1.0,2.500,2.625,전남,연산동


In [155]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [156]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/전남/전남_연산동_결측처리.csv',index=False)

## 장흥읍

In [157]:
df = pd.read_csv(path+"전남_장흥읍_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [158]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,0.4750,1.2250,58.625,66.125,5.9,-3.5,329.3750,297.6250,전남,장흥읍
1,2012-01-02,0.0,0.0,0.1125,-0.9250,66.000,57.125,5.1,-3.4,308.7500,321.3750,전남,장흥읍
2,2012-01-03,1.0,1.0,-1.0250,-0.6375,58.375,60.250,4.1,-3.6,321.8750,328.8750,전남,장흥읍
3,2012-01-04,0.0,0.0,-0.9375,-0.2625,64.375,63.500,5.1,-5.5,315.1250,289.0000,전남,장흥읍
4,2012-01-05,0.0,0.0,-0.1250,-0.0625,63.875,50.250,6.1,-4.7,341.5000,346.2500,전남,장흥읍
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,0.1000,0.3000,51.625,62.500,6.4,-3.9,2.7625,2.3000,전남,장흥읍
1823,2016-12-28,0.0,0.0,0.4750,0.2375,61.750,60.625,8.3,-2.9,2.2750,2.0875,전남,장흥읍
1824,2016-12-29,0.0,0.0,-0.1000,1.5875,58.625,68.500,11.2,-3.6,1.5000,1.6500,전남,장흥읍
1825,2016-12-30,0.0,0.0,1.4250,3.7375,68.500,66.500,12.0,-3.7,1.6500,1.7500,전남,장흥읍


In [159]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [160]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/전남/전남_장흥읍_결측처리.csv',index=False)

## 중마동

In [161]:
df = pd.read_csv(path+"전남_중마동_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [162]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,0.4875,1.4625,54.750,54.625,6.4,-4.0,322.125,291.625,전남,중마동
1,2012-01-02,0.0,0.0,0.5375,-1.0250,54.625,49.375,5.4,-3.6,288.000,312.875,전남,중마동
2,2012-01-03,0.0,0.0,-0.9750,-0.7125,45.500,50.500,4.0,-4.3,315.125,307.250,전남,중마동
3,2012-01-04,0.0,0.0,-0.9625,0.3125,47.250,48.750,6.5,-4.5,328.875,321.125,전남,중마동
4,2012-01-05,0.0,0.0,-0.6750,0.3500,51.375,47.250,6.5,-6.2,315.625,253.500,전남,중마동
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,2.3750,2.0000,47.500,48.125,8.0,-3.0,1.500,3.000,전남,중마동
1823,2016-12-28,0.0,0.0,2.0000,2.6250,47.500,50.000,9.0,-2.0,2.875,1.250,전남,중마동
1824,2016-12-29,0.0,0.0,2.5000,4.3750,50.625,63.125,11.0,-3.0,1.500,1.500,전남,중마동
1825,2016-12-30,0.0,0.0,4.3750,6.5000,63.750,61.250,12.0,0.0,1.250,1.625,전남,중마동


In [163]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [164]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/전남/전남_중마동_결측처리.csv',index=False)

## 중앙동

In [165]:
df = pd.read_csv(path+"전남_중앙동_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [166]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,0.9125,2.5000,51.875,50.000,6.0,-1.9,289.0000,292.5000,전남,중앙동
1,2012-01-02,0.0,0.0,1.1500,-0.8125,51.250,41.875,5.0,-2.9,315.0000,308.5000,전남,중앙동
2,2012-01-03,0.0,0.0,-0.8125,-0.2000,45.000,45.625,4.0,-2.9,315.0000,315.0000,전남,중앙동
3,2012-01-04,0.0,0.0,-0.2000,1.5250,45.625,48.750,6.0,-2.9,315.0000,315.0000,전남,중앙동
4,2012-01-05,0.0,0.0,1.1500,1.2750,45.000,42.000,7.0,-2.9,315.3750,337.5000,전남,중앙동
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,1.6875,1.8125,43.125,46.125,6.4,-2.5,2.8375,4.5500,전남,중앙동
1823,2016-12-28,0.0,0.0,2.0250,2.5375,45.500,44.500,8.3,-1.3,4.8250,2.2375,전남,중앙동
1824,2016-12-29,0.0,0.0,1.9875,4.9000,41.875,54.250,10.0,-2.0,2.8125,2.9375,전남,중앙동
1825,2016-12-30,0.0,0.0,4.4875,6.7500,54.250,53.125,10.9,1.0,2.8250,2.4625,전남,중앙동


In [167]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [168]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/전남/전남_중앙동_결측처리.csv',index=False)

## 진도읍

In [169]:
df = pd.read_csv(path+"전남_진도읍_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [170]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,0.9125,2.5000,51.875,50.000,6.0,-1.9,289.000,292.500,전남,진도읍
1,2012-01-02,0.0,0.0,1.1500,-0.8125,51.250,41.875,5.0,-2.9,315.000,308.500,전남,진도읍
2,2012-01-03,0.0,0.0,-0.8125,-0.2000,45.000,45.625,4.0,-2.9,315.000,315.000,전남,진도읍
3,2012-01-04,0.0,0.0,-0.2000,1.5250,45.625,48.750,6.0,-2.9,315.000,315.000,전남,진도읍
4,2012-01-05,0.0,0.0,1.1500,1.2750,45.000,42.000,7.0,-2.9,315.375,337.500,전남,진도읍
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,0.6250,1.3750,62.500,64.375,6.0,-2.0,3.000,5.375,전남,진도읍
1823,2016-12-28,0.0,0.0,1.1250,0.5000,64.375,65.000,8.0,-2.0,5.375,2.000,전남,진도읍
1824,2016-12-29,0.0,0.0,0.3750,1.8750,65.000,66.875,10.0,-3.0,2.250,1.500,전남,진도읍
1825,2016-12-30,0.0,0.0,2.7500,4.2500,66.875,65.625,11.0,-3.0,1.500,1.500,전남,진도읍


In [171]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [172]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/전남/전남_진도읍_결측처리.csv',index=False)

## 해남읍

In [173]:
df = pd.read_csv(path+"전남_해남읍_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [174]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,1.0,5.0,-0.2000,0.6375,59.625,73.000,4.8,-4.4,320.8750,288.6250,전남,해남읍
1,2012-01-02,0.0,0.0,-0.6125,-1.0875,70.000,61.250,4.6,-4.4,237.7500,327.1250,전남,해남읍
2,2012-01-03,1.0,5.0,-0.9000,-0.8625,63.500,64.500,3.8,-3.4,325.7500,332.7500,전남,해남읍
3,2012-01-04,1.0,1.0,-1.0375,-0.6000,65.000,64.000,4.7,-5.0,274.5000,213.0000,전남,해남읍
4,2012-01-05,0.0,0.0,-0.7250,-0.7875,63.500,50.250,5.8,-5.2,332.2500,338.0000,전남,해남읍
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-0.6500,0.3125,58.250,62.125,5.8,-3.8,1.8875,3.0500,전남,해남읍
1823,2016-12-28,0.0,0.0,0.4250,-0.7000,61.250,63.375,7.9,-3.6,2.9250,2.1000,전남,해남읍
1824,2016-12-29,0.0,0.0,-1.2125,0.5125,63.750,70.500,9.9,-5.1,1.5250,1.4375,전남,해남읍
1825,2016-12-30,0.0,0.0,0.4500,3.5500,70.500,71.375,10.8,-5.0,1.4375,1.9500,전남,해남읍


In [175]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [176]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/전남/전남_해남읍_결측처리.csv',index=False)

## 흑산면

In [177]:
df = pd.read_csv(path+"전남_흑산면_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [178]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,5.0,5.0,3.00,3.000,64.375,68.125,5.0,2.0,331.000,304.250,전남,흑산면
1,2012-01-02,0.0,0.0,2.50,0.250,68.750,63.125,4.0,-0.9,313.875,330.750,전남,흑산면
2,2012-01-03,5.0,5.0,0.25,2.125,70.000,60.000,3.0,-0.9,330.750,325.875,전남,흑산면
3,2012-01-04,0.0,0.0,1.50,3.000,63.750,59.375,5.0,1.0,324.625,333.125,전남,흑산면
4,2012-01-05,0.0,0.0,3.00,2.750,61.875,60.000,5.0,2.0,336.125,140.750,전남,흑산면
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,2.75,3.125,66.875,63.750,5.0,1.0,7.625,9.250,전남,흑산면
1823,2016-12-28,0.0,0.0,3.00,2.750,64.375,65.000,8.0,2.0,9.250,6.000,전남,흑산면
1824,2016-12-29,0.0,0.0,3.00,6.000,64.375,61.250,9.0,2.0,3.875,3.500,전남,흑산면
1825,2016-12-30,0.0,0.0,6.00,7.500,61.250,62.500,11.0,2.0,3.500,4.625,전남,흑산면


In [179]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [180]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/전남/전남_흑산면_결측처리.csv',index=False)

# 충북

## 강내면

In [181]:
df = pd.read_csv(path+"충북_강내면_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [182]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,-4.0000,-1.6375,61.750,71.375,2.0,-9.6,227.6250,256.3750,충북,강내면
1,2012-01-02,0.0,0.0,-2.2375,-3.5375,67.125,55.750,1.0,-6.1,290.7500,312.5000,충북,강내면
2,2012-01-03,0.0,0.0,-4.4750,-3.7250,57.500,57.125,0.0,-8.6,309.0000,264.1250,충북,강내면
3,2012-01-04,0.0,0.0,-4.4125,-2.0625,66.875,68.875,1.9,-8.5,294.8750,238.1250,충북,강내면
4,2012-01-05,0.0,0.0,-3.5875,-3.3375,55.500,53.250,2.3,-8.5,303.8750,202.2500,충북,강내면
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-3.8750,-2.3500,61.625,60.750,3.8,-10.1,0.6625,1.8125,충북,강내면
1823,2016-12-28,1.0,1.0,-2.4625,-3.1750,61.375,64.750,5.5,-9.4,1.7875,0.8875,충북,강내면
1824,2016-12-29,0.0,0.0,-2.2750,0.7750,59.250,74.625,8.6,-7.1,1.0000,1.0875,충북,강내면
1825,2016-12-30,0.0,0.0,0.7375,2.4375,76.750,75.875,9.4,-2.8,1.0625,0.8500,충북,강내면


In [183]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [184]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/충북/충북_강내면_결측처리.csv',index=False)

## 교현안림동

In [185]:
df = pd.read_csv(path+"충북_교현안림동_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [186]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,-5.1625,-2.8000,63.375,65.500,1.0,-10.9,172.125,222.125,충북,교현안림동
1,2012-01-02,0.0,0.0,-4.2875,-6.2750,65.625,53.500,1.0,-10.9,256.125,290.625,충북,교현안림동
2,2012-01-03,0.0,0.0,-5.5250,-6.0250,58.750,65.625,-0.9,-11.9,290.375,277.375,충북,교현안림동
3,2012-01-04,0.0,0.0,-7.2750,-4.7875,60.625,58.750,1.0,-12.9,244.875,159.125,충북,교현안림동
4,2012-01-05,0.0,0.0,-5.2875,-5.6625,60.625,62.125,2.0,-10.9,279.125,169.000,충북,교현안림동
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-2.3750,-2.7500,61.250,55.625,3.0,-9.0,1.375,1.750,충북,교현안림동
1823,2016-12-28,1.0,1.0,-2.6250,-2.5000,56.875,63.125,5.0,-9.0,3.750,1.250,충북,교현안림동
1824,2016-12-29,0.0,0.0,-3.3750,-0.8750,65.625,71.875,7.0,-10.0,1.125,1.250,충북,교현안림동
1825,2016-12-30,0.0,0.0,-1.0000,1.0000,73.125,77.500,9.0,-5.0,1.250,1.250,충북,교현안림동


In [187]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [188]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/충북/충북_교현안림동_결측처리.csv',index=False)

## 보은읍

In [189]:
df = pd.read_csv(path+"충북_보은읍_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [190]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,-4.6625,-2.1375,67.500,75.250,1.2,-11.5,183.3750,246.3750,충북,보은읍
1,2012-01-02,0.0,0.0,-2.5750,-4.0625,70.875,59.875,1.1,-7.4,280.3750,300.5000,충북,보은읍
2,2012-01-03,1.0,1.0,-4.7750,-5.0625,59.875,63.625,0.2,-10.6,301.3750,269.1250,충북,보은읍
3,2012-01-04,0.0,0.0,-5.5375,-3.7000,67.375,71.250,2.2,-10.6,293.2500,257.0000,충북,보은읍
4,2012-01-05,0.0,0.0,-4.7375,-4.6000,63.125,60.125,2.1,-10.6,297.5000,186.0000,충북,보은읍
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-3.7000,-2.7500,66.375,60.250,2.9,-10.2,1.0500,1.5250,충북,보은읍
1823,2016-12-28,1.0,1.0,-2.4625,-2.8750,62.000,68.625,4.7,-8.3,1.9125,1.2500,충북,보은읍
1824,2016-12-29,0.0,0.0,-3.3875,-0.6500,64.500,75.000,8.2,-9.7,1.2125,1.4125,충북,보은읍
1825,2016-12-30,0.0,0.0,-1.3625,0.4000,79.375,78.750,9.3,-6.8,1.3875,1.2875,충북,보은읍


In [191]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [192]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/충북/충북_보은읍_결측처리.csv',index=False)

## 복대1동

In [193]:
df = pd.read_csv(path+"충북_복대1동_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [194]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,-3.7875,-1.4375,59.625,72.125,2.0,-8.9,245.125,217.625,충북,복대1동
1,2012-01-02,0.0,0.0,-1.9125,-3.5250,64.625,53.375,1.0,-5.9,293.250,309.750,충북,복대1동
2,2012-01-03,0.0,0.0,-4.2750,-3.5375,53.750,53.250,0.0,-7.9,307.000,262.125,충북,복대1동
3,2012-01-04,0.0,0.0,-4.4000,-1.8125,68.750,71.250,2.0,-7.9,252.250,258.125,충북,복대1동
4,2012-01-05,0.0,0.0,-3.2875,-2.8000,51.500,47.875,2.0,-7.9,297.625,279.250,충북,복대1동
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-2.1250,-1.0000,60.625,51.250,3.0,-7.0,1.000,1.625,충북,복대1동
1823,2016-12-28,5.0,1.0,-1.6250,-1.6250,55.625,58.750,5.0,-7.0,1.875,1.375,충북,복대1동
1824,2016-12-29,0.0,0.0,-1.7500,1.1250,56.250,69.375,8.0,-6.0,1.125,1.375,충북,복대1동
1825,2016-12-30,0.0,0.0,1.1250,3.0000,72.500,71.875,9.0,-3.0,1.375,1.250,충북,복대1동


In [195]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [196]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/충북/충북_복대1동_결측처리.csv',index=False)

## 용두동

In [197]:
df = pd.read_csv(path+"충북_용두동_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [198]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,-6.5375,-4.6625,65.875,66.500,1.0,-12.9,164.875,194.750,충북,용두동
1,2012-01-02,0.0,0.0,-5.9000,-7.9000,63.875,53.875,0.0,-12.9,293.875,287.000,충북,용두동
2,2012-01-03,0.0,0.0,-7.1500,-7.7750,59.375,61.875,-1.9,-13.9,285.125,203.750,충북,용두동
3,2012-01-04,0.0,0.0,-9.6500,-6.2875,61.250,60.000,1.0,-16.9,218.250,199.625,충북,용두동
4,2012-01-05,0.0,0.0,-8.0375,-7.2875,64.000,61.875,1.0,-14.9,210.625,186.125,충북,용두동
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-5.0000,-4.1250,65.000,55.000,2.0,-13.0,1.500,2.000,충북,용두동
1823,2016-12-28,1.0,1.0,-4.2500,-4.7500,55.000,63.125,5.0,-11.0,2.000,1.500,충북,용두동
1824,2016-12-29,0.0,0.0,-5.3750,-2.0000,69.375,78.125,6.0,-12.0,1.375,1.500,충북,용두동
1825,2016-12-30,0.0,0.0,-2.0000,-0.2500,77.500,76.250,8.0,-6.0,1.500,1.500,충북,용두동


In [199]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [200]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/충북/충북_용두동_결측처리.csv',index=False)

## 추풍령면

In [201]:
df = pd.read_csv(path+"충북_추풍령면_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [202]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,-3.6625,-1.4375,60.375,67.625,1.0,-7.9,278.250,273.750,충북,추풍령면
1,2012-01-02,0.0,0.0,-2.4125,-3.7750,62.750,55.875,1.0,-6.9,280.500,298.000,충북,추풍령면
2,2012-01-03,1.0,1.0,-4.6500,-4.4000,55.500,54.375,-0.9,-7.9,297.875,290.250,충북,추풍령면
3,2012-01-04,0.0,0.0,-4.7750,-3.0500,68.125,71.250,2.0,-8.9,288.250,268.875,충북,추풍령면
4,2012-01-05,0.0,0.0,-3.4125,-3.4250,52.875,46.625,2.0,-8.9,288.375,277.000,충북,추풍령면
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-2.1250,-1.2500,56.875,54.375,3.0,-8.0,2.750,4.250,충북,추풍령면
1823,2016-12-28,5.0,1.0,-1.6250,-2.0000,58.750,59.375,5.0,-6.0,4.875,3.625,충북,추풍령면
1824,2016-12-29,0.0,0.0,-2.0000,0.1250,53.750,64.375,8.0,-6.0,3.250,2.250,충북,추풍령면
1825,2016-12-30,0.0,0.0,-0.1250,2.0000,71.875,70.625,9.0,-4.0,2.250,2.000,충북,추풍령면


In [203]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [204]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/충북/충북_추풍령면_결측처리.csv',index=False)

#울산

## 병영2동

In [205]:
df = pd.read_csv(path+"울산_병영2동_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [206]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,1.5375,2.000,48.625,53.000,6.6,-2.2,292.375,285.375,울산,병영2동
1,2012-01-02,0.0,0.0,1.4000,-0.250,50.750,40.375,5.7,-3.2,286.875,297.250,울산,병영2동
2,2012-01-03,0.0,0.0,-0.7375,0.250,37.250,38.375,4.6,-5.2,295.625,296.125,울산,병영2동
3,2012-01-04,0.0,0.0,-0.7250,1.400,38.875,46.375,5.7,-5.0,295.250,295.375,울산,병영2동
4,2012-01-05,0.0,0.0,0.4375,1.025,40.375,43.750,6.5,-4.7,297.625,297.125,울산,병영2동
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,2.8750,1.875,55.000,43.125,8.0,-2.0,2.875,3.125,울산,병영2동
1823,2016-12-28,0.0,0.0,2.2500,2.625,47.500,42.500,8.0,-2.0,2.750,2.250,울산,병영2동
1824,2016-12-29,0.0,0.0,2.3750,3.750,42.500,58.125,11.0,-2.0,2.375,2.500,울산,병영2동
1825,2016-12-30,0.0,0.0,3.8750,5.750,61.250,61.250,13.0,-1.0,1.875,1.375,울산,병영2동


In [207]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [208]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/울산/울산_병영2동_결측처리.csv',index=False)

# 전북

## 고창읍

In [209]:
df = pd.read_csv(path+"전북_고창읍_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [210]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,5.0,5.0,-1.3750,-0.1375,66.625,80.000,3.5,-6.1,315.1250,330.5000,전북,고창읍
1,2012-01-02,0.0,0.0,-0.7875,-2.9625,79.500,73.625,2.6,-4.7,260.7500,328.8750,전북,고창읍
2,2012-01-03,5.0,5.0,-2.6750,-1.9000,76.250,73.500,0.6,-5.4,316.7500,221.2500,전북,고창읍
3,2012-01-04,1.0,1.0,-2.7250,-1.2500,69.750,69.000,3.0,-6.8,287.2500,234.8750,전북,고창읍
4,2012-01-05,0.0,0.0,-2.6750,-2.0750,69.625,62.125,3.0,-8.5,327.6250,197.2500,전북,고창읍
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-2.1750,-1.1250,57.875,63.125,4.2,-6.3,2.0125,3.1625,전북,고창읍
1823,2016-12-28,5.0,1.0,-1.2625,-1.1875,66.375,66.125,7.1,-5.5,2.7125,1.6000,전북,고창읍
1824,2016-12-29,0.0,0.0,-1.1875,1.7750,63.875,71.625,9.4,-4.9,1.4625,1.5250,전북,고창읍
1825,2016-12-30,0.0,0.0,2.2875,3.6500,73.875,69.625,10.4,-0.9,1.6000,1.3125,전북,고창읍


In [211]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [212]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/전북/전북_고창읍_결측처리.csv',index=False)

## 구암동

In [213]:
df = pd.read_csv(path+"전북_구암동_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [214]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,1.0,1.0,-1.8625,-0.8250,66.375,76.625,1.9,-6.5,212.3750,204.000,전북,구암동
1,2012-01-02,0.0,0.0,-1.2750,-2.7625,74.750,63.500,2.1,-4.8,212.3750,319.875,전북,구암동
2,2012-01-03,5.0,5.0,-2.7375,-1.9250,67.000,68.375,1.0,-5.3,324.2500,320.375,전북,구암동
3,2012-01-04,0.0,0.0,-2.6750,-1.2375,70.500,73.250,2.9,-6.2,210.8750,163.625,전북,구암동
4,2012-01-05,0.0,0.0,-2.8375,-2.4375,65.000,66.000,2.9,-7.9,164.0000,82.125,전북,구암동
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-1.5625,-0.5375,63.625,67.375,3.2,-6.0,2.3750,2.425,전북,구암동
1823,2016-12-28,1.0,1.0,-0.7875,-0.9375,67.625,68.625,7.0,-4.5,2.3625,1.650,전북,구암동
1824,2016-12-29,0.0,0.0,-1.0000,2.4750,70.250,78.125,8.7,-4.6,1.5500,1.775,전북,구암동
1825,2016-12-30,1.0,1.0,2.7000,4.2500,80.750,75.375,9.9,0.1,1.6250,1.700,전북,구암동


In [215]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [216]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/전북/전북_구암동_결측처리.csv',index=False)

## 대산면

In [217]:
df = pd.read_csv(path+"전북_대산면_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [218]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,5.0,5.0,-1.4875,-0.4125,68.625,80.500,3.5,-6.4,2.4875,2.3250,전북,대산면
1,2012-01-02,0.0,0.0,-0.8750,-3.1500,80.875,73.250,2.4,-5.0,3.2875,4.7375,전북,대산면
2,2012-01-03,5.0,5.0,-2.8250,-1.7875,80.000,75.000,1.3,-5.2,5.1000,2.1375,전북,대산면
3,2012-01-04,1.0,1.0,-1.8375,-0.8500,67.750,68.250,3.0,-5.5,2.9375,2.3625,전북,대산면
4,2012-01-05,0.0,0.0,-2.0500,-1.8750,68.500,63.625,3.0,-7.8,2.1250,1.8375,전북,대산면
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-1.8875,-1.0375,61.125,64.500,4.0,-6.3,2.1625,3.3375,전북,대산면
1823,2016-12-28,1.0,1.0,-1.3750,-1.1875,65.875,65.875,7.7,-6.0,2.9875,1.8375,전북,대산면
1824,2016-12-29,0.0,0.0,-1.0000,1.7250,63.250,70.875,9.4,-4.8,1.4875,1.5875,전북,대산면
1825,2016-12-30,0.0,0.0,2.0500,3.7000,72.375,68.375,10.5,-1.7,1.6625,1.2625,전북,대산면


In [219]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [220]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/전북/전북_대산면_결측처리.csv',index=False)

## 덕진동

In [221]:
df = pd.read_csv(path+"전북_덕진동_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [222]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,5.0,5.0,-2.0625,-0.8125,72.500,71.625,3.0,-5.9,260.750,198.000,전북,덕진동
1,2012-01-02,0.0,0.0,-0.8125,-2.1625,71.250,63.750,3.0,-4.9,209.500,321.000,전북,덕진동
2,2012-01-03,1.0,1.0,-2.5250,-1.9375,56.875,59.875,2.0,-5.9,321.875,247.625,전북,덕진동
3,2012-01-04,0.0,0.0,-3.4125,-1.4375,72.625,73.750,3.0,-6.9,287.500,183.625,전북,덕진동
4,2012-01-05,0.0,0.0,-2.1875,-2.4375,70.000,64.375,4.0,-6.9,188.750,202.500,전북,덕진동
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-1.2500,-0.3750,63.125,65.625,5.0,-5.0,1.500,2.125,전북,덕진동
1823,2016-12-28,1.0,5.0,-0.8750,-0.5000,64.375,65.625,8.0,-4.0,2.125,1.375,전북,덕진동
1824,2016-12-29,0.0,0.0,-0.2500,2.6250,65.625,67.500,10.0,-5.0,1.375,1.625,전북,덕진동
1825,2016-12-30,0.0,0.0,2.6250,4.2500,67.500,68.750,11.0,-2.0,1.375,1.500,전북,덕진동


In [223]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [224]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/전북/전북_덕진동_결측처리.csv',index=False)

## 도통동

In [225]:
df = pd.read_csv(path+"전북_도통동_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [226]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,1.0,5.0,-5.0000,-2.3750,74.875,74.125,2.8,-11.4,167.5000,189.7500,전북,도통동
1,2012-01-02,0.0,0.0,-2.7375,-3.6875,77.125,71.500,2.9,-7.0,219.2500,315.1250,전북,도통동
2,2012-01-03,1.0,1.0,-3.9000,-4.3750,64.125,68.625,0.9,-9.2,314.2500,258.8750,전북,도통동
3,2012-01-04,0.0,0.0,-6.1625,-3.8000,75.875,80.125,2.8,-12.0,258.3750,215.0000,전북,도통동
4,2012-01-05,0.0,0.0,-5.2125,-4.8000,74.125,66.875,4.0,-11.3,265.5000,265.8750,전북,도통동
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-2.0125,-1.1375,67.875,66.875,5.0,-7.7,1.2250,2.0125,전북,도통동
1823,2016-12-28,0.0,0.0,-1.3250,-1.7625,66.000,69.000,7.1,-6.4,2.0875,1.2750,전북,도통동
1824,2016-12-29,0.0,0.0,-2.0250,0.6875,68.750,73.125,9.4,-7.8,1.3000,1.2750,전북,도통동
1825,2016-12-30,0.0,0.0,0.7125,1.8875,74.125,73.625,10.4,-4.5,1.1125,0.9750,전북,도통동


In [227]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [228]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/전북/전북_도통동_결측처리.csv',index=False)

## 상교동

In [229]:
df = pd.read_csv(path+"전북_상교동_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [230]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,5.0,5.0,-0.9500,0.1625,66.250,85.625,3.0,-4.9,315.000,298.125,전북,상교동
1,2012-01-02,0.0,0.0,-0.5750,-2.2750,77.500,76.250,2.0,-3.9,304.250,315.000,전북,상교동
2,2012-01-03,5.0,5.0,-2.4000,-1.6875,63.750,65.625,1.0,-4.9,329.750,179.500,전북,상교동
3,2012-01-04,1.0,5.0,-3.9125,-1.5625,68.125,69.000,4.0,-7.9,119.625,128.750,전북,상교동
4,2012-01-05,0.0,0.0,-4.0500,-2.8125,71.250,61.250,3.0,-8.9,315.000,270.000,전북,상교동
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-1.7500,-0.7500,59.375,63.125,3.0,-6.0,2.125,1.375,전북,상교동
1823,2016-12-28,5.0,5.0,-1.5000,-0.7500,65.000,65.625,7.0,-5.0,1.375,1.000,전북,상교동
1824,2016-12-29,0.0,0.0,-0.7500,2.7500,65.625,70.625,10.0,-5.0,1.000,1.125,전북,상교동
1825,2016-12-30,0.0,0.0,2.7500,4.5000,70.625,68.125,11.0,0.0,1.250,1.125,전북,상교동


In [231]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [232]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/전북/전북_상교동_결측처리.csv',index=False)

## 순창읍

In [233]:
df = pd.read_csv(path+"전북_순창읍_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [234]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,5.0,5.0,-3.1750,-2.0625,64.375,83.750,3.0,-7.9,315.000,298.125,전북,순창읍
1,2012-01-02,0.0,0.0,-2.0625,-3.5250,76.250,73.125,2.0,-5.9,304.250,315.000,전북,순창읍
2,2012-01-03,5.0,5.0,-3.6500,-3.6625,65.000,66.875,0.0,-6.9,325.000,211.500,전북,순창읍
3,2012-01-04,0.0,0.0,-5.2750,-4.0625,70.375,70.500,4.0,-9.9,165.750,117.625,전북,순창읍
4,2012-01-05,0.0,0.0,-3.9250,-3.3000,68.750,61.250,4.0,-9.9,315.000,270.000,전북,순창읍
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-2.5000,-1.5000,62.500,69.375,4.0,-7.0,1.875,2.750,전북,순창읍
1823,2016-12-28,0.0,0.0,-2.1250,-1.6250,69.375,65.625,7.0,-7.0,2.750,1.750,전북,순창읍
1824,2016-12-29,0.0,0.0,-2.2500,0.1250,66.250,73.125,10.0,-8.0,1.750,1.625,전북,순창읍
1825,2016-12-30,0.0,0.0,0.5000,2.5000,73.125,70.625,11.0,-5.0,1.500,1.500,전북,순창읍


In [235]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [236]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/전북/전북_순창읍_결측처리.csv',index=False)

## 임실읍

In [237]:
df = pd.read_csv(path+"전북_임실읍_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [238]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,5.0,5.0,-5.2875,-3.2875,63.125,83.125,2.0,-10.9,315.000,298.125,전북,임실읍
1,2012-01-02,0.0,0.0,-3.7875,-5.1500,78.750,76.250,1.0,-8.9,304.250,315.000,전북,임실읍
2,2012-01-03,5.0,5.0,-5.5250,-5.0375,65.000,67.500,0.0,-8.9,318.125,312.125,전북,임실읍
3,2012-01-04,0.0,0.0,-6.6500,-5.0500,70.375,73.000,3.0,-12.9,181.000,139.500,전북,임실읍
4,2012-01-05,0.0,0.0,-5.5375,-4.9250,67.500,56.250,3.0,-11.9,315.000,270.000,전북,임실읍
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-3.7500,-3.0000,63.750,70.625,4.0,-10.0,1.375,2.000,전북,임실읍
1823,2016-12-28,0.0,0.0,-3.1250,-2.7500,70.000,69.375,6.0,-7.0,2.000,1.125,전북,임실읍
1824,2016-12-29,0.0,0.0,-3.2500,-1.2500,68.750,73.125,9.0,-10.0,1.125,1.125,전북,임실읍
1825,2016-12-30,0.0,0.0,-0.5000,1.3750,73.125,72.500,10.0,-6.0,0.625,0.500,전북,임실읍


In [239]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [240]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/전북/전북_임실읍_결측처리.csv',index=False)

## 장수읍

In [241]:
df = pd.read_csv(path+"전북_장수읍_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [242]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,5.0,5.0,-6.5375,-3.1750,72.500,74.500,1.0,-11.9,149.875,251.625,전북,장수읍
1,2012-01-02,0.0,0.0,-4.0375,-5.1500,77.750,72.875,1.0,-7.9,258.500,309.875,전북,장수읍
2,2012-01-03,1.0,1.0,-5.9000,-5.9000,62.000,66.000,0.0,-9.9,309.500,293.000,전북,장수읍
3,2012-01-04,0.0,0.0,-7.6500,-6.0375,76.500,77.875,1.0,-12.9,157.500,157.500,전북,장수읍
4,2012-01-05,0.0,0.0,-6.7875,-6.0375,74.375,67.875,2.0,-11.9,112.500,135.000,전북,장수읍
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-4.0000,-2.5000,66.875,69.375,3.0,-10.0,1.500,2.625,전북,장수읍
1823,2016-12-28,0.0,0.0,-3.3750,-3.2500,68.750,66.875,5.0,-7.0,2.750,1.625,전북,장수읍
1824,2016-12-29,0.0,0.0,-4.2500,-1.0000,66.875,70.625,8.0,-10.0,1.625,1.625,전북,장수읍
1825,2016-12-30,0.0,0.0,-0.3750,0.3750,73.125,73.125,9.0,-7.0,1.250,1.500,전북,장수읍


In [243]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [244]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/전북/전북_장수읍_결측처리.csv',index=False)

## 행안면

In [245]:
df = pd.read_csv(path+"전북_행안면_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [246]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,5.0,5.0,-0.9375,-0.2000,63.125,85.625,3.0,-4.9,315.000,298.125,전북,행안면
1,2012-01-02,0.0,0.0,-0.2000,-2.0250,79.375,76.250,2.0,-2.9,304.250,315.000,전북,행안면
2,2012-01-03,5.0,5.0,-2.1500,-1.4250,63.750,69.375,1.0,-3.9,333.625,179.125,전북,행안면
3,2012-01-04,1.0,5.0,-3.1625,-1.8125,71.375,74.875,4.0,-6.9,139.375,187.375,전북,행안면
4,2012-01-05,0.0,0.0,-3.6875,-2.3125,72.500,66.250,3.0,-8.9,315.000,337.500,전북,행안면
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-1.5000,-0.3750,68.750,67.500,4.0,-6.0,2.000,1.875,전북,행안면
1823,2016-12-28,5.0,5.0,-0.5000,-0.8750,68.750,70.000,7.0,-5.0,1.875,1.500,전북,행안면
1824,2016-12-29,0.0,0.0,-0.7500,2.1250,71.250,76.250,9.0,-5.0,1.500,1.500,전북,행안면
1825,2016-12-30,1.0,1.0,2.5000,3.7500,78.125,75.000,10.0,0.0,1.500,1.500,전북,행안면


In [247]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [248]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/전북/전북_행안면_결측처리.csv',index=False)

# 제주

## 건입동

In [249]:
df = pd.read_csv(path+"제주_건입동_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [250]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,5.0,0.0,5.000,5.875,56.500,60.375,8.0,4.0,334.750,316.625,제주,건입동
1,2012-01-02,1.0,0.0,5.250,2.125,61.125,60.625,7.0,1.0,314.375,331.250,제주,건입동
2,2012-01-03,5.0,0.0,2.750,3.375,63.750,57.500,5.0,1.0,315.000,315.000,제주,건입동
3,2012-01-04,5.0,0.0,3.375,5.000,54.375,54.500,7.0,2.0,331.875,295.750,제주,건입동
4,2012-01-05,0.0,0.0,5.375,4.875,55.625,53.750,8.0,3.0,336.875,350.750,제주,건입동
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,4.500,5.250,53.750,57.500,8.0,4.0,6.250,9.125,제주,건입동
1823,2016-12-28,0.0,0.0,5.250,4.875,56.250,55.000,10.0,4.0,5.750,3.750,제주,건입동
1824,2016-12-29,0.0,0.0,5.125,6.750,60.625,54.375,13.0,4.0,3.250,3.375,제주,건입동
1825,2016-12-30,0.0,0.0,6.750,9.000,58.750,58.125,13.0,3.0,3.125,3.125,제주,건입동


In [251]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [252]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/제주/제주_건입동_결측처리.csv',index=False)

## 성산읍

In [253]:
df = pd.read_csv(path+"제주_성산읍_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [254]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,5.0,0.0,4.4125,5.4625,60.750,65.750,7.9,1.9,335.500,314.8750,제주,성산읍
1,2012-01-02,1.0,0.0,4.8875,2.5750,67.000,64.125,8.1,1.0,317.125,331.1250,제주,성산읍
2,2012-01-03,5.0,0.0,2.4250,3.7000,64.500,58.875,6.0,1.2,330.625,334.5000,제주,성산읍
3,2012-01-04,5.0,5.0,3.5000,5.0875,60.125,62.000,8.0,0.1,332.500,338.3750,제주,성산읍
4,2012-01-05,0.0,0.0,4.0875,4.0250,66.250,55.375,7.9,1.9,341.000,342.0000,제주,성산읍
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,4.5500,4.9625,60.000,66.500,8.2,2.3,3.875,5.6500,제주,성산읍
1823,2016-12-28,0.0,0.0,5.0875,4.8750,66.375,62.750,10.1,2.3,5.350,3.3000,제주,성산읍
1824,2016-12-29,0.0,0.0,4.7875,6.6875,63.125,66.375,13.9,2.6,4.075,3.3375,제주,성산읍
1825,2016-12-30,0.0,0.0,6.6875,9.1125,67.625,65.625,13.5,2.5,3.350,3.0625,제주,성산읍


In [255]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [256]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/제주/제주_성산읍_결측처리.csv',index=False)

## 정방동

In [257]:
df = pd.read_csv(path+"제주_정방동_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [258]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,6.0000,6.7500,60.750,62.250,11.0,3.0,321.3750,297.6250,제주,정방동
1,2012-01-02,0.0,0.0,5.8750,3.1250,62.875,56.875,10.0,2.0,299.8750,316.6250,제주,정방동
2,2012-01-03,0.0,0.0,3.2500,4.6250,57.875,57.625,8.0,1.0,318.8750,279.5000,제주,정방동
3,2012-01-04,0.0,0.0,4.6250,6.3750,58.125,57.375,10.0,2.0,320.3750,297.2500,제주,정방동
4,2012-01-05,0.0,0.0,5.8750,6.1250,57.375,56.375,11.0,2.0,285.3750,305.6250,제주,정방동
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,5.9000,5.0125,55.875,58.875,10.9,2.1,2.3125,3.0000,제주,정방동
1823,2016-12-28,0.0,0.0,5.4375,7.1250,59.375,58.375,11.2,2.9,3.4625,2.2375,제주,정방동
1824,2016-12-29,0.0,0.0,7.2500,7.8125,59.125,58.750,14.3,3.2,2.1250,1.9750,제주,정방동
1825,2016-12-30,0.0,0.0,7.8250,10.6875,58.750,59.625,14.3,4.5,2.1000,1.6625,제주,정방동


In [259]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [260]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/제주/제주_정방동_결측처리.csv',index=False)

## 한경면

In [261]:
df = pd.read_csv(path+"제주_한경면_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [262]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,5.0,0.0,5.0375,6.0125,57.000,60.125,8.5,3.6,334.6250,320.1250,제주,한경면
1,2012-01-02,1.0,0.0,5.3000,2.2875,61.750,59.000,6.9,0.9,319.3750,335.5000,제주,한경면
2,2012-01-03,1.0,0.0,2.8000,3.3875,63.250,57.750,5.0,1.0,315.5000,314.0000,제주,한경면
3,2012-01-04,1.0,0.0,3.3000,4.8250,56.375,56.500,7.5,1.6,335.0000,345.7500,제주,한경면
4,2012-01-05,0.0,0.0,5.1250,4.4125,55.125,56.375,7.9,2.7,341.3750,221.8750,제주,한경면
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,5.0750,5.8500,56.125,58.250,7.9,4.0,9.0875,13.7375,제주,한경면
1823,2016-12-28,0.0,0.0,5.9125,5.2750,57.125,59.375,9.9,4.0,12.5875,8.5750,제주,한경면
1824,2016-12-29,0.0,0.0,5.6250,7.5250,59.000,59.625,13.1,3.6,6.0000,5.9250,제주,한경면
1825,2016-12-30,0.0,0.0,7.4250,9.6250,61.250,60.000,13.2,3.3,5.0125,4.3875,제주,한경면


In [263]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [264]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/제주/제주_한경면_결측처리.csv',index=False)

# 충남

## 금산읍

In [265]:
df = pd.read_csv(path+"충남_금산읍_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [266]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,-4.9125,-2.4375,70.875,74.500,2.0,-11.9,107.3750,136.2500,충남,금산읍
1,2012-01-02,0.0,0.0,-2.6750,-3.9000,70.000,58.375,1.0,-6.9,169.2500,301.0000,충남,금산읍
2,2012-01-03,0.0,0.0,-4.6500,-5.1625,57.625,63.750,1.0,-10.9,302.3750,250.2500,충남,금산읍
3,2012-01-04,0.0,0.0,-5.4000,-3.6875,66.875,71.250,2.0,-9.9,286.3750,286.0000,충남,금산읍
4,2012-01-05,0.0,0.0,-5.1625,-4.5500,65.875,61.500,3.0,-10.9,199.2500,138.8750,충남,금산읍
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-4.2500,-2.7500,63.375,65.000,4.0,-11.0,0.5250,1.6125,충남,금산읍
1823,2016-12-28,1.0,1.0,-2.8750,-4.0000,66.875,71.250,7.0,-10.0,1.4875,0.4125,충남,금산읍
1824,2016-12-29,0.0,0.0,-3.3750,-0.2500,65.000,81.875,9.0,-9.0,0.3875,0.4250,충남,금산읍
1825,2016-12-30,0.0,0.0,-0.2500,1.5000,85.000,80.000,9.0,-4.0,0.4500,0.2750,충남,금산읍


In [267]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [268]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/충남/충남_급산읍_결측처리.csv',index=False)

## 대천5동

In [269]:
df = pd.read_csv(path+"충남_대천5동_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [270]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,5.0,5.0,-1.5125,-0.6250,60.625,70.625,2.3,-5.5,321.2500,299.3750,충남,대천5동
1,2012-01-02,0.0,0.0,-0.5375,-2.3875,66.625,58.500,2.3,-3.6,305.6250,321.1250,충남,대천5동
2,2012-01-03,5.0,5.0,-2.6875,-1.9625,60.500,62.375,1.2,-4.5,327.6250,324.5000,충남,대천5동
3,2012-01-04,0.0,0.0,-2.8000,-1.2625,63.625,65.250,2.2,-5.7,278.7500,339.2500,충남,대천5동
4,2012-01-05,0.0,0.0,-2.4250,-1.7750,62.750,53.500,3.1,-6.6,207.2500,350.5000,충남,대천5동
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-1.4750,-0.2000,61.375,59.250,2.9,-5.7,1.4625,2.5875,충남,대천5동
1823,2016-12-28,1.0,1.0,-0.6250,-0.5375,61.000,64.375,6.7,-4.4,2.3875,1.4000,충남,대천5동
1824,2016-12-29,0.0,0.0,-0.5750,3.2750,64.250,66.375,8.2,-4.4,1.5500,1.8500,충남,대천5동
1825,2016-12-30,1.0,1.0,3.6625,4.7375,68.625,65.875,9.2,0.4,1.4875,1.7250,충남,대천5동


In [271]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [272]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/충남/충남_대천5동_결측처리.csv',index=False)

## 병천면

In [273]:
df = pd.read_csv(path+"충남_병천면_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [274]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,-4.2875,-2.1875,63.250,71.500,2.0,-10.2,192.000,257.250,충남,병천면
1,2012-01-02,0.0,0.0,-2.6250,-4.0375,68.875,58.125,1.0,-7.2,264.625,316.375,충남,병천면
2,2012-01-03,1.0,5.0,-4.8125,-4.1875,60.000,58.875,0.0,-9.3,312.500,269.250,충남,병천면
3,2012-01-04,0.0,0.0,-4.9125,-2.6500,68.375,69.500,1.8,-9.9,295.750,261.750,충남,병천면
4,2012-01-05,0.0,0.0,-4.2125,-3.8250,56.375,52.750,2.3,-8.9,269.125,219.750,충남,병천면
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-4.0000,-2.6250,65.750,59.625,3.0,-10.0,1.250,2.125,충남,병천면
1823,2016-12-28,1.0,1.0,-2.7500,-2.5000,59.625,63.250,6.0,-10.0,2.125,2.000,충남,병천면
1824,2016-12-29,0.0,0.0,-2.7500,0.7500,64.375,73.750,8.0,-9.0,2.000,1.625,충남,병천면
1825,2016-12-30,0.0,0.0,0.8750,2.7500,73.750,77.500,10.0,-3.0,1.250,1.250,충남,병천면


In [275]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [276]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/충남/충남_병천면_결측처리.csv',index=False)

## 부여읍

In [277]:
df = pd.read_csv(path+"충남_부여읍_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [278]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,5.0,5.0,-2.5625,-1.8125,67.750,74.125,2.0,-7.9,315.000,298.125,충남,부여읍
1,2012-01-02,0.0,0.0,-1.0625,-2.7875,69.875,56.625,2.0,-5.9,298.125,315.000,충남,부여읍
2,2012-01-03,5.0,5.0,-3.0250,-1.9375,56.875,64.375,2.0,-6.9,324.500,324.625,충남,부여읍
3,2012-01-04,0.0,0.0,-3.7875,-2.3125,63.750,67.500,3.0,-7.9,315.000,334.000,충남,부여읍
4,2012-01-05,0.0,0.0,-3.6875,-2.8125,65.000,58.875,3.0,-8.9,331.875,348.750,충남,부여읍
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-2.6250,-2.1250,66.375,64.625,4.0,-8.0,1.000,1.375,충남,부여읍
1823,2016-12-28,1.0,1.0,-2.1250,-2.0000,64.625,69.375,7.0,-8.0,1.375,1.375,충남,부여읍
1824,2016-12-29,0.0,0.0,-2.2500,0.8750,68.125,72.500,9.0,-8.0,1.375,1.500,충남,부여읍
1825,2016-12-30,0.0,0.0,0.6250,2.3750,73.750,74.875,10.0,-3.0,1.125,0.900,충남,부여읍


In [279]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [280]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/충남/충남_부여읍_결측처리.csv',index=False)

## 수석동

In [281]:
df = pd.read_csv(path+"충남_수석동_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [282]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,5.0,5.0,-2.1500,-0.7250,69.500,74.750,2.2,-5.5,313.1250,292.8750,충남,수석동
1,2012-01-02,0.0,0.0,-2.5500,-3.2125,67.875,59.250,1.1,-5.6,276.5000,324.5000,충남,수석동
2,2012-01-03,5.0,5.0,-3.4750,-2.8375,64.250,62.375,0.2,-7.4,326.2500,322.8750,충남,수석동
3,2012-01-04,0.0,0.0,-3.2375,-1.2375,63.125,66.625,2.2,-6.3,230.3750,244.8750,충남,수석동
4,2012-01-05,0.0,0.0,-2.6625,-2.9875,67.625,60.750,3.2,-7.6,286.5000,139.2500,충남,수석동
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-1.7125,-0.6125,64.375,60.000,3.2,-6.2,1.3875,3.4000,충남,수석동
1823,2016-12-28,1.0,1.0,-0.9500,-0.5625,60.750,69.500,7.3,-5.2,3.0875,1.3000,충남,수석동
1824,2016-12-29,0.0,0.0,-1.3125,2.0250,72.000,81.500,9.0,-6.7,1.5125,1.8875,충남,수석동
1825,2016-12-30,1.0,1.0,2.1500,3.4375,79.000,75.875,9.3,-2.2,1.4500,1.8875,충남,수석동


In [283]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [284]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/충남/충남_수석동_결측처리.csv',index=False)

## 홍북읍

In [285]:
df = pd.read_csv(path+"충남_홍북읍_결측처리.csv",index_col=0) 
data = df.reset_index(drop = True)

rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train = train.interpolate()
test = test.interpolate()

all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 

df_final = pd.concat([df3, rain_snow], axis=1)

df_final['date'] = all_ymd
df_final = df_final[['date', 'rainfall_24', 'snow_24', 'temp_24', 'temp_48', 'humid_24','humid_48', 'max_temp_24', 'min_temp_24', 'wind_24', 'wind_48','area','region']]

In [286]:
df_final

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,1.0,1.0,-3.3625,-1.3625,68.000,72.500,1.9,-7.8,320.6250,297.5000,충남,홍북읍
1,2012-01-02,0.0,0.0,-3.7500,-4.1125,69.125,60.250,0.1,-6.7,315.3750,324.2500,충남,홍북읍
2,2012-01-03,5.0,5.0,-4.6750,-3.8875,64.250,63.125,0.4,-9.0,322.3750,284.7500,충남,홍북읍
3,2012-01-04,0.0,0.0,-4.0000,-2.3375,64.000,66.875,2.0,-8.7,223.6250,276.2500,충남,홍북읍
4,2012-01-05,0.0,0.0,-4.4000,-4.1375,66.125,62.125,2.9,-8.5,308.3750,339.2500,충남,홍북읍
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2016-12-27,0.0,0.0,-3.3250,-1.9250,65.750,64.250,2.7,-8.0,1.0250,2.0125,충남,홍북읍
1823,2016-12-28,1.0,1.0,-2.3500,-2.5250,64.250,70.250,6.4,-7.9,1.7875,1.1250,충남,홍북읍
1824,2016-12-29,0.0,0.0,-2.6500,0.8500,70.250,78.625,8.8,-8.0,1.2000,1.2875,충남,홍북읍
1825,2016-12-30,1.0,1.0,1.1875,2.6375,79.000,76.625,9.0,-2.3,0.9000,1.4375,충남,홍북읍


In [287]:
df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [288]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/충남/충남_홍북읍_결측처리.csv',index=False)

# 강원

## 강원 - 갈말읍

In [289]:
## csv 파일들을 DataFrame으로 불러와서 concat
df = pd.read_csv(path+"강원_갈말읍_결측처리.csv",index_col=0)
    
data = df.reset_index(drop = True)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,1.0,1.0,-6.900,-6.0375,68.125,68.750,0.0,-12.9,220.375,213.125,강원,갈말읍
1,2012-01-02,0.0,0.0,-7.900,-9.7750,72.500,63.125,-2.9,-15.9,235.875,260.250,강원,갈말읍
2,2012-01-03,0.0,0.0,-10.025,-9.7750,65.625,66.250,-1.9,-16.9,263.500,219.875,강원,갈말읍
3,2012-01-04,0.0,0.0,-9.025,-6.4000,65.625,62.500,0.0,-16.9,277.625,217.625,강원,갈말읍
4,2012-01-05,0.0,0.0,-9.025,-8.9000,68.125,67.500,0.0,-14.9,264.500,204.000,강원,갈말읍


In [290]:
# 강수, 적설은 na 0으로 채움
rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)


# train / test 나눠서 interpolate
train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train=train.interpolate()
test=test.interpolate()

# 데이터 합치기
all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 
df3['date']=all_ymd

df_final = pd.concat([df3, rain_snow], axis=1)
df_final = df_final[['date','rainfall_24','snow_24','temp_24','temp_48','humid_24','humid_48','max_temp_24','min_temp_24','wind_24','wind_48','area','region']]
df_final

df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [291]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/강원/강원_갈말읍_결측처리.csv',index=False)

## 강원 - 대관령면

In [292]:
## csv 파일들을 DataFrame으로 불러와서 concat
df = pd.read_csv(path+"강원_대관령면_결측처리.csv",index_col=0)
    
data = df.reset_index(drop = True)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,5.0,5.0,-7.9375,-8.5000,70.375,66.375,-1.1,NaN,263.500,257.375,강원,대관령면
1,2012-01-02,0.0,0.0,-9.0375,-10.6750,70.500,63.875,-4.1,NaN,262.000,247.625,강원,대관령면
2,2012-01-03,0.0,0.0,-11.2500,-10.5375,65.000,59.375,-4.2,NaN,249.000,245.500,강원,대관령면
3,2012-01-04,0.0,0.0,-10.5625,-8.7750,61.625,65.625,-3.1,NaN,269.375,269.000,강원,대관령면
4,2012-01-05,0.0,0.0,-10.2875,-10.2625,59.625,57.125,-2.1,NaN,276.500,180.875,강원,대관령면


In [293]:
data.loc[data['date']=='2016-01-01']

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
1461,2016-01-01,0.0,0.0,0.0375,0.0625,74.875,76.25,5.2,-2.6,7.75,5.4875,강원,대관령면


In [294]:
data['min_temp_24'][0] = -2.6

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [295]:
data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,5.0,5.0,-7.9375,-8.5000,70.375,66.375,-1.1,-2.6,263.500,257.375,강원,대관령면
1,2012-01-02,0.0,0.0,-9.0375,-10.6750,70.500,63.875,-4.1,NaN,262.000,247.625,강원,대관령면
2,2012-01-03,0.0,0.0,-11.2500,-10.5375,65.000,59.375,-4.2,NaN,249.000,245.500,강원,대관령면
3,2012-01-04,0.0,0.0,-10.5625,-8.7750,61.625,65.625,-3.1,NaN,269.375,269.000,강원,대관령면
4,2012-01-05,0.0,0.0,-10.2875,-10.2625,59.625,57.125,-2.1,NaN,276.500,180.875,강원,대관령면


In [296]:
# 강수, 적설은 na 0으로 채움
rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)


# train / test 나눠서 interpolate
train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train=train.interpolate()
test=test.interpolate()

# 데이터 합치기
all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 
df3['date']=all_ymd

df_final = pd.concat([df3, rain_snow], axis=1)
df_final = df_final[['date','rainfall_24','snow_24','temp_24','temp_48','humid_24','humid_48','max_temp_24','min_temp_24','wind_24','wind_48','area','region']]
df_final

df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [297]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/강원/강원_대관령면_결측처리.csv',index=False)

## 강원 - 명륜1동

In [298]:
## csv 파일들을 DataFrame으로 불러와서 concat
df = pd.read_csv(path+"강원_명륜1동_결측처리.csv",index_col=0)
    
data = df.reset_index(drop = True)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,1.0,1.0,-4.7875,-4.1625,56.875,66.250,1.0,-9.9,218.125,227.50,강원,명륜1동
1,2012-01-02,0.0,0.0,-4.2875,-5.1500,66.250,50.000,0.0,-9.9,244.000,301.00,강원,명륜1동
2,2012-01-03,0.0,0.0,-6.5250,-5.5375,51.875,51.875,0.0,-11.9,282.750,244.50,강원,명륜1동
3,2012-01-04,0.0,0.0,-6.7750,-4.1625,54.375,52.500,2.0,-11.9,264.625,204.75,강원,명륜1동
4,2012-01-05,0.0,0.0,-6.2875,-6.0375,55.625,55.000,2.0,-10.9,289.875,202.75,강원,명륜1동


In [299]:
# 강수, 적설은 na 0으로 채움
rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)


# train / test 나눠서 interpolate
train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train=train.interpolate()
test=test.interpolate()

# 데이터 합치기
all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 
df3['date']=all_ymd

df_final = pd.concat([df3, rain_snow], axis=1)
df_final = df_final[['date','rainfall_24','snow_24','temp_24','temp_48','humid_24','humid_48','max_temp_24','min_temp_24','wind_24','wind_48','area','region']]
df_final

df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [300]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/강원/강원_명륜1동_결측처리.csv',index=False)

## 강원 - 사천면

In [301]:
## csv 파일들을 DataFrame으로 불러와서 concat
df = pd.read_csv(path+"강원_사천면_결측처리.csv",index_col=0)
    
data = df.reset_index(drop = True)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,25.0,20.0,-0.8125,-1.4375,60.000,42.500,3.0,-3.9,232.000,265.250,강원,사천면
1,2012-01-02,0.0,0.0,-1.9250,-3.3000,52.125,47.250,3.0,-6.9,270.375,253.000,강원,사천면
2,2012-01-03,0.0,0.0,-3.5500,-2.4375,47.500,40.125,3.0,-7.9,255.625,248.125,강원,사천면
3,2012-01-04,0.0,0.0,-2.4250,-1.1875,35.000,41.875,4.0,-7.9,250.000,281.375,강원,사천면
4,2012-01-05,0.0,0.0,-2.3000,-1.8000,40.000,42.500,4.0,-4.9,266.875,283.000,강원,사천면


In [302]:
# 강수, 적설은 na 0으로 채움
rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)


# train / test 나눠서 interpolate
train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train=train.interpolate()
test=test.interpolate()

# 데이터 합치기
all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 
df3['date']=all_ymd

df_final = pd.concat([df3, rain_snow], axis=1)
df_final = df_final[['date','rainfall_24','snow_24','temp_24','temp_48','humid_24','humid_48','max_temp_24','min_temp_24','wind_24','wind_48','area','region']]
df_final

df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [303]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/강원/강원_사천면_결측처리.csv',index=False)

## 강원 - 송정동

In [304]:
## csv 파일들을 DataFrame으로 불러와서 concat
df = pd.read_csv(path+"강원_송정동_결측처리.csv",index_col=0)
    
data = df.reset_index(drop = True)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,25.0,20.0,0.3000,-0.1875,60.000,42.500,5.0,-1.9,232.000,265.250,강원,송정동
1,2012-01-02,0.0,0.0,0.4250,-1.1875,49.875,44.500,5.0,-3.9,277.000,257.875,강원,송정동
2,2012-01-03,0.0,0.0,-1.5625,-0.0750,47.500,40.125,5.0,-4.9,262.750,253.375,강원,송정동
3,2012-01-04,0.0,0.0,-0.5750,0.1750,39.000,48.625,5.0,-4.9,253.250,294.750,강원,송정동
4,2012-01-05,0.0,0.0,-0.4375,-0.0750,40.000,42.500,5.0,-2.9,266.875,283.000,강원,송정동


In [305]:
# 강수, 적설은 na 0으로 채움
rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)


# train / test 나눠서 interpolate
train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train=train.interpolate()
test=test.interpolate()

# 데이터 합치기
all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 
df3['date']=all_ymd

df_final = pd.concat([df3, rain_snow], axis=1)
df_final = df_final[['date','rainfall_24','snow_24','temp_24','temp_48','humid_24','humid_48','max_temp_24','min_temp_24','wind_24','wind_48','area','region']]
df_final

df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [306]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/강원/강원_송정동_결측처리.csv',index=False)

## 강원 - 신북읍

In [307]:
## csv 파일들을 DataFrame으로 불러와서 concat
df = pd.read_csv(path+"강원_신북읍_결측처리.csv",index_col=0)
    
data = df.reset_index(drop = True)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,1.0,1.0,-5.6250,-4.8250,65.875,70.375,2.2,-11.9,313.00,292.750,강원,신북읍
1,2012-01-02,0.0,0.0,-6.5000,-7.6125,66.875,57.250,0.1,-12.3,191.25,179.750,강원,신북읍
2,2012-01-03,0.0,0.0,-8.1000,-7.8750,56.625,58.750,-0.8,-13.9,237.25,255.500,강원,신북읍
3,2012-01-04,0.0,0.0,-8.6500,-5.7125,63.750,64.750,1.9,-14.9,270.00,242.875,강원,신북읍
4,2012-01-05,0.0,0.0,-8.7625,-8.3625,66.625,67.250,2.0,-14.4,304.00,287.375,강원,신북읍


In [308]:
# 강수, 적설은 na 0으로 채움
rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)


# train / test 나눠서 interpolate
train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train=train.interpolate()
test=test.interpolate()

# 데이터 합치기
all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 
df3['date']=all_ymd

df_final = pd.concat([df3, rain_snow], axis=1)
df_final = df_final[['date','rainfall_24','snow_24','temp_24','temp_48','humid_24','humid_48','max_temp_24','min_temp_24','wind_24','wind_48','area','region']]
df_final

df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [309]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/강원/강원_신북읍_결측처리.csv',index=False)

## 강원 - 신사우동

In [310]:
## csv 파일들을 DataFrame으로 불러와서 concat
df = pd.read_csv(path+"강원_신사우동_결측처리.csv",index_col=0)
    
data = df.reset_index(drop = True)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,1.0,1.0,-5.3000,-4.6625,65.625,70.625,2.0,-10.9,315.000,292.50,강원,신사우동
1,2012-01-02,0.0,0.0,-6.0375,-7.4000,66.250,56.875,0.0,-11.9,191.250,225.00,강원,신사우동
2,2012-01-03,0.0,0.0,-7.7750,-7.2750,54.375,56.250,-0.9,-12.9,281.250,258.75,강원,신사우동
3,2012-01-04,0.0,0.0,-8.1500,-5.1625,61.250,61.875,2.0,-13.9,258.750,225.00,강원,신사우동
4,2012-01-05,0.0,0.0,-8.1625,-7.6625,63.125,62.500,2.0,-12.9,309.375,292.50,강원,신사우동


In [311]:
# 강수, 적설은 na 0으로 채움
rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)


# train / test 나눠서 interpolate
train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train=train.interpolate()
test=test.interpolate()

# 데이터 합치기
all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 
df3['date']=all_ymd

df_final = pd.concat([df3, rain_snow], axis=1)
df_final = df_final[['date','rainfall_24','snow_24','temp_24','temp_48','humid_24','humid_48','max_temp_24','min_temp_24','wind_24','wind_48','area','region']]
df_final

df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [312]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/강원/강원_신사우동_결측처리.csv',index=False)

## 강원 - 영월읍

In [313]:
## csv 파일들을 DataFrame으로 불러와서 concat
df = pd.read_csv(path+"강원_영월읍_결측처리.csv",index_col=0)
    
data = df.reset_index(drop = True)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,5.0,5.0,-4.9375,-4.0250,60.875,62.875,1.8,-11.2,201.000,130.875,강원,영월읍
1,2012-01-02,0.0,0.0,-5.0000,-7.7875,61.125,49.500,1.8,-12.1,272.625,278.125,강원,영월읍
2,2012-01-03,0.0,0.0,-6.5500,-6.6375,45.000,56.500,-0.2,-12.2,277.000,241.875,강원,영월읍
3,2012-01-04,0.0,0.0,-7.1375,-4.7625,57.750,60.875,1.7,-13.2,162.250,201.625,강원,영월읍
4,2012-01-05,0.0,0.0,-5.8000,-5.8875,56.500,56.000,2.8,-12.5,144.000,179.125,강원,영월읍


In [314]:
# 강수, 적설은 na 0으로 채움
rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)


# train / test 나눠서 interpolate
train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train=train.interpolate()
test=test.interpolate()

# 데이터 합치기
all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 
df3['date']=all_ymd

df_final = pd.concat([df3, rain_snow], axis=1)
df_final = df_final[['date','rainfall_24','snow_24','temp_24','temp_48','humid_24','humid_48','max_temp_24','min_temp_24','wind_24','wind_48','area','region']]
df_final

df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [315]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/강원/강원_영월읍_결측처리.csv',index=False)

## 강원 - 인제읍

In [316]:
## csv 파일들을 DataFrame으로 불러와서 concat
df = pd.read_csv(path+"강원_인제읍_결측처리.csv",index_col=0)
    
data = df.reset_index(drop = True)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,1.0,1.0,-5.6625,-5.0375,66.250,67.500,2.0,-9.9,180.0,180.0,강원,인제읍
1,2012-01-02,0.0,0.0,-6.5250,-7.9000,62.500,56.875,-0.9,-12.9,270.0,315.0,강원,인제읍
2,2012-01-03,0.0,0.0,-8.4000,-8.4000,53.750,56.250,-1.9,-14.9,180.0,180.0,강원,인제읍
3,2012-01-04,0.0,0.0,-8.6500,-6.1625,61.250,61.875,1.0,-14.9,180.0,180.0,강원,인제읍
4,2012-01-05,0.0,0.0,-8.9125,-8.2875,64.375,60.625,1.0,-13.9,202.5,247.5,강원,인제읍


In [317]:
# 강수, 적설은 na 0으로 채움
rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)


# train / test 나눠서 interpolate
train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train=train.interpolate()
test=test.interpolate()

# 데이터 합치기
all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 
df3['date']=all_ymd

df_final = pd.concat([df3, rain_snow], axis=1)
df_final = df_final[['date','rainfall_24','snow_24','temp_24','temp_48','humid_24','humid_48','max_temp_24','min_temp_24','wind_24','wind_48','area','region']]
df_final

df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [318]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/강원/강원_인제읍_결측처리.csv',index=False)

## 강원 - 정선읍

In [319]:
## csv 파일들을 DataFrame으로 불러와서 concat
df = pd.read_csv(path+"강원_정선읍_결측처리.csv",index_col=0)
    
data = df.reset_index(drop = True)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,5.0,5.0,-4.9375,-4.3000,65.75,62.625,1.6,-11.3,92.500,183.625,강원,정선읍
1,2012-01-02,0.0,0.0,-5.3375,-8.0500,67.00,53.125,1.6,-12.4,257.500,253.125,강원,정선읍
2,2012-01-03,0.0,0.0,-7.2000,-6.8000,44.25,55.875,-0.4,-12.5,253.500,231.750,강원,정선읍
3,2012-01-04,0.0,0.0,-7.6500,-5.2750,58.25,61.000,1.5,-13.5,235.375,211.125,강원,정선읍
4,2012-01-05,0.0,0.0,-6.6875,-5.8125,56.50,55.375,2.4,-12.7,250.250,89.500,강원,정선읍


In [320]:
# 강수, 적설은 na 0으로 채움
rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)


# train / test 나눠서 interpolate
train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train=train.interpolate()
test=test.interpolate()

# 데이터 합치기
all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 
df3['date']=all_ymd

df_final = pd.concat([df3, rain_snow], axis=1)
df_final = df_final[['date','rainfall_24','snow_24','temp_24','temp_48','humid_24','humid_48','max_temp_24','min_temp_24','wind_24','wind_48','area','region']]
df_final

df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [321]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/강원/강원_정선읍_결측처리.csv',index=False)

## 강원 - 중앙동

In [322]:
## csv 파일들을 DataFrame으로 불러와서 concat
df = pd.read_csv(path+"강원_중앙동_결측처리.csv",index_col=0)
    
data = df.reset_index(drop = True)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,25.0,20.0,0.1750,-0.1875,60.000,42.500,4.0,-1.9,232.000,265.25,강원,중앙동
1,2012-01-02,0.0,0.0,0.1750,-1.8000,44.750,39.750,5.0,-4.9,273.625,257.25,강원,중앙동
2,2012-01-03,0.0,0.0,-1.9375,-0.6875,47.500,40.125,4.0,-5.9,260.250,253.00,강원,중앙동
3,2012-01-04,0.0,0.0,-0.5750,0.1750,31.875,41.250,5.0,-4.9,253.750,284.75,강원,중앙동
4,2012-01-05,0.0,0.0,-0.5625,0.1750,40.000,42.500,5.0,-2.9,266.875,283.00,강원,중앙동


In [323]:
# 강수, 적설은 na 0으로 채움
rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)


# train / test 나눠서 interpolate
train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train=train.interpolate()
test=test.interpolate()

# 데이터 합치기
all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 
df3['date']=all_ymd

df_final = pd.concat([df3, rain_snow], axis=1)
df_final = df_final[['date','rainfall_24','snow_24','temp_24','temp_48','humid_24','humid_48','max_temp_24','min_temp_24','wind_24','wind_48','area','region']]
df_final

df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [324]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/강원/강원_중앙동_결측처리.csv',index=False)

## 강원 - 토성면

In [325]:
## csv 파일들을 DataFrame으로 불러와서 concat
df = pd.read_csv(path+"강원_토성면_결측처리.csv",index_col=0)
    
data = df.reset_index(drop = True)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,10.0,10.0,0.0500,-0.6875,45.000,45.000,3.0,-2.9,236.250,270.000,강원,토성면
1,2012-01-02,0.0,0.0,-0.9375,-2.3000,50.625,46.875,3.0,-4.9,256.500,246.750,강원,토성면
2,2012-01-03,0.0,0.0,-2.0625,-1.4375,43.125,39.375,3.0,-5.9,292.500,258.750,강원,토성면
3,2012-01-04,0.0,0.0,-1.6875,-0.9375,36.250,43.750,4.0,-5.9,270.000,298.125,강원,토성면
4,2012-01-05,0.0,0.0,-1.1875,-0.4375,34.375,33.750,5.0,-3.9,287.375,268.500,강원,토성면


In [326]:
# 강수, 적설은 na 0으로 채움
rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)


# train / test 나눠서 interpolate
train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train=train.interpolate()
test=test.interpolate()

# 데이터 합치기
all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 
df3['date']=all_ymd

df_final = pd.concat([df3, rain_snow], axis=1)
df_final = df_final[['date','rainfall_24','snow_24','temp_24','temp_48','humid_24','humid_48','max_temp_24','min_temp_24','wind_24','wind_48','area','region']]
df_final

df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [327]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/강원/강원_토성면_결측처리.csv',index=False)

## 강원 - 홍천읍

In [328]:
## csv 파일들을 DataFrame으로 불러와서 concat
df = pd.read_csv(path+"강원_홍천읍_결측처리.csv",index_col=0)
    
data = df.reset_index(drop = True)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,1.0,1.0,-5.4125,-4.7875,61.250,66.250,2.0,-10.9,315.000,292.500,강원,홍천읍
1,2012-01-02,0.0,0.0,-5.9125,-7.5250,66.250,56.875,1.0,-12.9,191.625,225.000,강원,홍천읍
2,2012-01-03,0.0,0.0,-8.1500,-8.1500,54.375,56.250,-0.9,-13.9,281.250,303.750,강원,홍천읍
3,2012-01-04,0.0,0.0,-8.4000,-5.4125,61.250,61.875,1.0,-13.9,157.500,157.500,강원,홍천읍
4,2012-01-05,0.0,0.0,-8.0375,-8.0375,58.125,55.625,2.0,-13.9,163.125,162.375,강원,홍천읍


In [329]:
# 강수, 적설은 na 0으로 채움
rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)


# train / test 나눠서 interpolate
train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train=train.interpolate()
test=test.interpolate()

# 데이터 합치기
all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 
df3['date']=all_ymd

df_final = pd.concat([df3, rain_snow], axis=1)
df_final = df_final[['date','rainfall_24','snow_24','temp_24','temp_48','humid_24','humid_48','max_temp_24','min_temp_24','wind_24','wind_48','area','region']]
df_final

df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [330]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/강원/강원_홍천읍_결측처리.csv',index=False)

## 강원 - 황지동

In [331]:
## csv 파일들을 DataFrame으로 불러와서 concat
df = pd.read_csv(path+"강원_황지동_결측처리.csv",index_col=0)
    
data = df.reset_index(drop = True)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,5.0,5.0,-4.9125,-5.1500,58.750,52.500,2.0,-9.9,274.125,265.000,강원,황지동
1,2012-01-02,0.0,0.0,-5.5250,-7.9000,51.875,46.875,0.0,-12.9,268.000,261.750,강원,황지동
2,2012-01-03,0.0,0.0,-8.0250,-7.5250,50.000,47.500,-0.9,-12.9,262.500,259.750,강원,황지동
3,2012-01-04,0.0,0.0,-8.0250,-5.2875,47.500,51.875,0.0,-13.9,260.000,276.875,강원,황지동
4,2012-01-05,0.0,0.0,-7.1500,-6.0375,46.250,48.750,1.0,-11.9,274.000,283.125,강원,황지동


In [332]:
# 강수, 적설은 na 0으로 채움
rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)


# train / test 나눠서 interpolate
train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train=train.interpolate()
test=test.interpolate()

# 데이터 합치기
all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 
df3['date']=all_ymd

df_final = pd.concat([df3, rain_snow], axis=1)
df_final = df_final[['date','rainfall_24','snow_24','temp_24','temp_48','humid_24','humid_48','max_temp_24','min_temp_24','wind_24','wind_48','area','region']]
df_final

df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [333]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/강원/강원_황지동_결측처리.csv',index=False)

# 경기

## 경기 - 문산읍

In [334]:
## csv 파일들을 DataFrame으로 불러와서 concat
df = pd.read_csv(path+"경기_문산읍_결측처리.csv",index_col=0)
    
data = df.reset_index(drop = True)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,1.0,1.0,-6.3000,-4.8625,70.000,72.250,1.0,-12.2,254.625,164.625,경기,문산읍
1,2012-01-02,0.0,0.0,-5.4625,-6.6500,63.000,52.750,-0.8,-10.6,235.250,302.625,경기,문산읍
2,2012-01-03,0.0,0.0,-8.0750,-7.7125,53.375,60.500,-0.7,-14.5,306.500,259.625,경기,문산읍
3,2012-01-04,0.0,0.0,-7.8750,-4.8375,71.375,66.750,1.0,-14.5,260.250,246.000,경기,문산읍
4,2012-01-05,0.0,0.0,-5.9875,-5.5875,65.000,62.375,2.2,-12.4,322.750,243.250,경기,문산읍


In [335]:
# 강수, 적설은 na 0으로 채움
rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)


# train / test 나눠서 interpolate
train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train=train.interpolate()
test=test.interpolate()

# 데이터 합치기
all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 
df3['date']=all_ymd

df_final = pd.concat([df3, rain_snow], axis=1)
df_final = df_final[['date','rainfall_24','snow_24','temp_24','temp_48','humid_24','humid_48','max_temp_24','min_temp_24','wind_24','wind_48','area','region']]
df_final

df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [336]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/경기/경기_문산읍_결측처리.csv',index=False)

## 경기 - 부발읍

In [337]:
## csv 파일들을 DataFrame으로 불러와서 concat
df = pd.read_csv(path+"경기_부발읍_결측처리.csv",index_col=0)
    
data = df.reset_index(drop = True)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,5.0,1.0,-5.2000,-3.9625,59.125,63.875,0.9,-10.8,283.750,281.875,경기,부발읍
1,2012-01-02,0.0,0.0,-5.5250,-6.2500,71.000,44.375,-0.9,-10.5,189.000,298.250,경기,부발읍
2,2012-01-03,0.0,0.0,-6.7875,-6.3625,50.500,55.625,-0.1,-12.9,308.875,297.750,경기,부발읍
3,2012-01-04,0.0,0.0,-6.4375,-3.3875,57.500,56.875,2.1,-12.8,263.500,244.000,경기,부발읍
4,2012-01-05,0.0,0.0,-5.3375,-5.2750,49.125,59.875,1.9,-10.8,303.000,63.625,경기,부발읍


In [338]:
# 강수, 적설은 na 0으로 채움
rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)


# train / test 나눠서 interpolate
train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train=train.interpolate()
test=test.interpolate()

# 데이터 합치기
all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 
df3['date']=all_ymd

df_final = pd.concat([df3, rain_snow], axis=1)
df_final = df_final[['date','rainfall_24','snow_24','temp_24','temp_48','humid_24','humid_48','max_temp_24','min_temp_24','wind_24','wind_48','area','region']]
df_final

df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [339]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/경기/경기_부발읍_결측처리.csv',index=False)

## 경기 - 생연1동

In [340]:
## csv 파일들을 DataFrame으로 불러와서 concat
df = pd.read_csv(path+"경기_생연1동_결측처리.csv",index_col=0)
    
data = df.reset_index(drop = True)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,1.0,1.0,-6.0375,-4.7875,62.500,64.375,1.0,-11.9,202.500,194.000,경기,생연1동
1,2012-01-02,0.0,0.0,-5.4000,-6.6500,61.250,51.250,-0.9,-9.9,249.125,291.000,경기,생연1동
2,2012-01-03,0.0,0.0,-8.1500,-7.2750,52.875,56.625,-0.9,-12.9,306.250,296.625,경기,생연1동
3,2012-01-04,0.0,0.0,-7.0250,-4.2875,61.250,60.625,2.0,-12.9,241.250,239.500,경기,생연1동
4,2012-01-05,0.0,0.0,-5.9125,-5.6625,55.625,55.000,2.0,-11.9,324.875,286.875,경기,생연1동


In [341]:
# 강수, 적설은 na 0으로 채움
rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)


# train / test 나눠서 interpolate
train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train=train.interpolate()
test=test.interpolate()

# 데이터 합치기
all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 
df3['date']=all_ymd

df_final = pd.concat([df3, rain_snow], axis=1)
df_final = df_final[['date','rainfall_24','snow_24','temp_24','temp_48','humid_24','humid_48','max_temp_24','min_temp_24','wind_24','wind_48','area','region']]
df_final

df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [342]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/경기/경기_생연1동_결측처리.csv',index=False)

## 경기 - 양평읍

In [343]:
## csv 파일들을 DataFrame으로 불러와서 concat
df = pd.read_csv(path+"경기_양평읍_결측처리.csv",index_col=0)
    
data = df.reset_index(drop = True)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,5.0,1.0,-5.1625,-3.7875,59.375,63.750,1.0,-9.9,281.250,281.250,경기,양평읍
1,2012-01-02,0.0,0.0,-5.0250,-5.9000,71.250,45.000,-0.9,-9.9,267.625,297.625,경기,양평읍
2,2012-01-03,0.0,0.0,-6.5250,-6.4000,51.500,56.875,0.0,-12.9,306.750,298.125,경기,양평읍
3,2012-01-04,0.0,0.0,-6.5250,-3.4375,58.750,55.000,2.0,-12.9,258.750,253.125,경기,양평읍
4,2012-01-05,0.0,0.0,-5.6625,-5.1625,49.375,60.000,2.0,-10.9,303.375,61.875,경기,양평읍


In [344]:
# 강수, 적설은 na 0으로 채움
rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)


# train / test 나눠서 interpolate
train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train=train.interpolate()
test=test.interpolate()

# 데이터 합치기
all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 
df3['date']=all_ymd

df_final = pd.concat([df3, rain_snow], axis=1)
df_final = df_final[['date','rainfall_24','snow_24','temp_24','temp_48','humid_24','humid_48','max_temp_24','min_temp_24','wind_24','wind_48','area','region']]
df_final

df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [345]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/경기/경기_양평읍_결측처리.csv',index=False)

## 경기 - 평동

In [346]:
## csv 파일들을 DataFrame으로 불러와서 concat
df = pd.read_csv(path+"경기_평동_결측처리.csv",index_col=0)
    
data = df.reset_index(drop = True)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,1.0,1.0,-3.7875,-3.0375,62.125,66.000,1.0,-7.9,183.875,260.500,경기,평동
1,2012-01-02,0.0,0.0,-3.2875,-4.5250,66.000,53.125,-0.9,-6.9,176.250,312.500,경기,평동
2,2012-01-03,0.0,0.0,-5.7750,-4.2875,53.500,58.250,0.0,-8.9,310.250,253.875,경기,평동
3,2012-01-04,0.0,0.0,-4.4125,-2.0625,58.125,57.500,2.0,-9.9,258.125,237.250,경기,평동
4,2012-01-05,0.0,0.0,-4.0375,-4.0375,55.375,53.000,2.0,-8.9,332.375,344.125,경기,평동


In [347]:
# 강수, 적설은 na 0으로 채움
rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)


# train / test 나눠서 interpolate
train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train=train.interpolate()
test=test.interpolate()

# 데이터 합치기
all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 
df3['date']=all_ymd

df_final = pd.concat([df3, rain_snow], axis=1)
df_final = df_final[['date','rainfall_24','snow_24','temp_24','temp_48','humid_24','humid_48','max_temp_24','min_temp_24','wind_24','wind_48','area','region']]
df_final

df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [348]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/경기/경기_평동_결측처리.csv',index=False)

# 경남

## 경남 - 가포동

In [349]:
## csv 파일들을 DataFrame으로 불러와서 concat
df = pd.read_csv(path+"경남_가포동_결측처리.csv",index_col=0)
    
data = df.reset_index(drop = True)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,1.1500,2.3875,46.250,53.750,7.0,-1.9,335.500,287.25,경남,가포동
1,2012-01-02,0.0,0.0,1.1625,0.3125,39.375,38.125,6.0,-2.9,348.750,348.75,경남,가포동
2,2012-01-03,0.0,0.0,-0.5625,0.1625,31.875,33.750,4.0,-3.9,315.000,315.00,경남,가포동
3,2012-01-04,0.0,0.0,-0.2000,0.9000,41.875,44.375,6.0,-3.9,315.000,315.00,경남,가포동
4,2012-01-05,0.0,0.0,1.6500,2.2750,38.125,36.250,7.0,-3.9,322.375,348.75,경남,가포동


In [350]:
# 강수, 적설은 na 0으로 채움
rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)


# train / test 나눠서 interpolate
train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train=train.interpolate()
test=test.interpolate()

# 데이터 합치기
all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 
df3['date']=all_ymd

df_final = pd.concat([df3, rain_snow], axis=1)
df_final = df_final[['date','rainfall_24','snow_24','temp_24','temp_48','humid_24','humid_48','max_temp_24','min_temp_24','wind_24','wind_48','area','region']]
df_final

df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [351]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/경남/경남_가포동_결측처리.csv',index=False)

## 경남 - 거창읍

In [352]:
## csv 파일들을 DataFrame으로 불러와서 concat
df = pd.read_csv(path+"경남_거창읍_결측처리.csv",index_col=0)
    
data = df.reset_index(drop = True)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,-3.3125,-1.1875,57.500,65.000,5.0,-8.9,306.00,292.500,경남,거창읍
1,2012-01-02,0.0,0.0,-1.5625,-3.2875,60.625,45.625,4.0,-5.9,294.00,306.625,경남,거창읍
2,2012-01-03,1.0,1.0,-3.5375,-4.0500,51.250,55.625,3.0,-9.9,315.00,303.750,경남,거창읍
3,2012-01-04,0.0,0.0,-4.5500,-3.0625,51.875,60.000,4.0,-9.9,302.25,285.250,경남,거창읍
4,2012-01-05,0.0,0.0,-4.1750,-3.5625,58.750,48.750,5.0,-10.9,303.75,320.625,경남,거창읍


In [353]:
# 강수, 적설은 na 0으로 채움
rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)


# train / test 나눠서 interpolate
train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train=train.interpolate()
test=test.interpolate()

# 데이터 합치기
all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 
df3['date']=all_ymd

df_final = pd.concat([df3, rain_snow], axis=1)
df_final = df_final[['date','rainfall_24','snow_24','temp_24','temp_48','humid_24','humid_48','max_temp_24','min_temp_24','wind_24','wind_48','area','region']]
df_final

df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [354]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/경남/경남_거창읍_결측처리.csv',index=False)

## 경남 - 내이동

In [355]:
## csv 파일들을 DataFrame으로 불러와서 concat
df = pd.read_csv(path+"경남_내이동_결측처리.csv",index_col=0)
    
data = df.reset_index(drop = True)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,-0.5625,0.2875,48.750,53.750,6.0,-6.9,295.250,288.500,경남,내이동
1,2012-01-02,0.0,0.0,0.6750,-0.8125,50.000,39.375,6.0,-5.9,299.500,310.875,경남,내이동
2,2012-01-03,0.0,0.0,-1.5625,-1.5625,36.250,40.625,5.0,-7.9,310.375,318.250,경남,내이동
3,2012-01-04,0.0,0.0,-2.1875,-0.4500,38.125,46.875,6.0,-8.9,299.125,297.625,경남,내이동
4,2012-01-05,0.0,0.0,-1.9375,-1.5625,42.500,43.750,7.0,-9.9,302.625,302.625,경남,내이동


In [356]:
# 강수, 적설은 na 0으로 채움
rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)


# train / test 나눠서 interpolate
train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train=train.interpolate()
test=test.interpolate()

# 데이터 합치기
all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 
df3['date']=all_ymd

df_final = pd.concat([df3, rain_snow], axis=1)
df_final = df_final[['date','rainfall_24','snow_24','temp_24','temp_48','humid_24','humid_48','max_temp_24','min_temp_24','wind_24','wind_48','area','region']]
df_final

df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [357]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/경남/경남_내이동_결측처리.csv',index=False)

## 경남 - 동면

In [358]:
## csv 파일들을 DataFrame으로 불러와서 concat
df = pd.read_csv(path+"경남_동면_결측처리.csv",index_col=0)
    
data = df.reset_index(drop = True)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,1.2875,1.725000e+00,46.125,51.750,6.7,-3.1,294.500,287.250,경남,동면
1,2012-01-02,0.0,0.0,1.2625,-2.775558e-17,47.375,35.875,5.6,-3.2,294.000,307.000,경남,동면
2,2012-01-03,0.0,0.0,0.0500,2.250000e-01,35.750,35.375,4.8,-4.9,305.500,310.625,경남,동면
3,2012-01-04,0.0,0.0,-0.3375,1.012500e+00,34.250,44.125,5.8,-4.3,300.875,298.125,경남,동면
4,2012-01-05,0.0,0.0,0.4375,1.175000e+00,38.000,40.750,6.9,-4.5,304.750,307.875,경남,동면


In [359]:
# 강수, 적설은 na 0으로 채움
rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)


# train / test 나눠서 interpolate
train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train=train.interpolate()
test=test.interpolate()

# 데이터 합치기
all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 
df3['date']=all_ymd

df_final = pd.concat([df3, rain_snow], axis=1)
df_final = df_final[['date','rainfall_24','snow_24','temp_24','temp_48','humid_24','humid_48','max_temp_24','min_temp_24','wind_24','wind_48','area','region']]
df_final

df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [360]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/경남/경남_동면_결측처리.csv',index=False)

## 경남 - 명곡동

In [361]:
## csv 파일들을 DataFrame으로 불러와서 concat
df = pd.read_csv(path+"경남_명곡동_결측처리.csv",index_col=0)
    
data = df.reset_index(drop = True)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,0.8875,1.7750,48.500,58.125,6.3,-3.5,242.875,286.875,경남,명곡동
1,2012-01-02,0.0,0.0,0.5375,-0.3875,43.375,38.500,6.1,-3.7,171.500,82.125,경남,명곡동
2,2012-01-03,0.0,0.0,-1.0125,-0.5875,31.625,35.500,4.0,-5.5,317.750,318.000,경남,명곡동
3,2012-01-04,0.0,0.0,-1.0500,-0.1250,42.875,45.000,6.0,-4.8,317.750,317.250,경남,명곡동
4,2012-01-05,0.0,0.0,0.3000,0.8125,38.625,37.625,7.0,-4.8,326.500,304.000,경남,명곡동


In [362]:
# 강수, 적설은 na 0으로 채움
rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)


# train / test 나눠서 interpolate
train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train=train.interpolate()
test=test.interpolate()

# 데이터 합치기
all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 
df3['date']=all_ymd

df_final = pd.concat([df3, rain_snow], axis=1)
df_final = df_final[['date','rainfall_24','snow_24','temp_24','temp_48','humid_24','humid_48','max_temp_24','min_temp_24','wind_24','wind_48','area','region']]
df_final

df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [363]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/경남/경남_명곡동_결측처리.csv',index=False)

## 경남 - 부원동

In [364]:
## csv 파일들을 DataFrame으로 불러와서 concat
df = pd.read_csv(path+"경남_부원동_결측처리.csv",index_col=0)
    
data = df.reset_index(drop = True)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,9.500000e-01,1.750,41.750,45.750,6.7,-3.9,319.125,287.250,경남,부원동
1,2012-01-02,0.0,0.0,1.037500e+00,-0.150,46.625,35.500,5.1,-3.9,294.250,313.375,경남,부원동
2,2012-01-03,0.0,0.0,-3.875000e-01,-0.200,33.625,36.875,4.9,-5.2,312.000,318.750,경남,부원동
3,2012-01-04,0.0,0.0,-4.625000e-01,0.300,35.000,42.500,6.0,-4.1,317.750,306.000,경남,부원동
4,2012-01-05,0.0,0.0,-5.551115e-17,0.825,35.125,36.125,6.9,-5.0,320.625,286.250,경남,부원동


In [365]:
# 강수, 적설은 na 0으로 채움
rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)


# train / test 나눠서 interpolate
train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train=train.interpolate()
test=test.interpolate()

# 데이터 합치기
all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 
df3['date']=all_ymd

df_final = pd.concat([df3, rain_snow], axis=1)
df_final = df_final[['date','rainfall_24','snow_24','temp_24','temp_48','humid_24','humid_48','max_temp_24','min_temp_24','wind_24','wind_48','area','region']]
df_final

df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [366]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/경남/경남_부원동_결측처리.csv',index=False)

## 경남 - 산청읍

In [367]:
## csv 파일들을 DataFrame으로 불러와서 concat
df = pd.read_csv(path+"경남_산청읍_결측처리.csv",index_col=0)
    
data = df.reset_index(drop = True)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,-1.1875,-0.0750,53.125,51.250,5.0,-6.9,268.500,313.250,경남,산청읍
1,2012-01-02,0.0,0.0,-0.4500,-1.7875,53.125,40.625,4.0,-4.9,291.875,309.500,경남,산청읍
2,2012-01-03,1.0,1.0,-1.5500,-1.5625,41.250,42.500,4.0,-5.9,309.250,294.875,경남,산청읍
3,2012-01-04,0.0,0.0,-2.3000,-1.4375,44.375,48.750,5.0,-5.9,292.250,278.000,경남,산청읍
4,2012-01-05,0.0,0.0,-0.9375,-1.4500,46.875,44.375,6.0,-8.9,319.875,142.750,경남,산청읍


In [368]:
# 강수, 적설은 na 0으로 채움
rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)


# train / test 나눠서 interpolate
train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train=train.interpolate()
test=test.interpolate()

# 데이터 합치기
all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 
df3['date']=all_ymd

df_final = pd.concat([df3, rain_snow], axis=1)
df_final = df_final[['date','rainfall_24','snow_24','temp_24','temp_48','humid_24','humid_48','max_temp_24','min_temp_24','wind_24','wind_48','area','region']]
df_final

df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [369]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/경남/경남_산청읍_결측처리.csv',index=False)

## 경남 - 의령읍

In [370]:
## csv 파일들을 DataFrame으로 불러와서 concat
df = pd.read_csv(path+"경남_의령읍_결측처리.csv",index_col=0)
    
data = df.reset_index(drop = True)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,-1.3750,0.1125,61.125,67.250,6.1,-8.2,285.375,287.750,경남,의령읍
1,2012-01-02,0.0,0.0,-1.3750,-2.1375,60.500,52.875,5.8,-7.0,255.750,341.000,경남,의령읍
2,2012-01-03,0.0,0.0,-2.5875,-2.2375,37.750,46.125,3.2,-9.0,314.625,311.875,경남,의령읍
3,2012-01-04,0.0,0.0,-3.1500,-2.0000,56.625,57.875,5.9,-8.5,313.500,311.000,경남,의령읍
4,2012-01-05,0.0,0.0,-3.1750,-2.4250,57.000,57.500,6.9,-10.3,320.000,170.875,경남,의령읍


In [371]:
# 강수, 적설은 na 0으로 채움
rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)


# train / test 나눠서 interpolate
train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train=train.interpolate()
test=test.interpolate()

# 데이터 합치기
all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 
df3['date']=all_ymd

df_final = pd.concat([df3, rain_snow], axis=1)
df_final = df_final[['date','rainfall_24','snow_24','temp_24','temp_48','humid_24','humid_48','max_temp_24','min_temp_24','wind_24','wind_48','area','region']]
df_final

df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [372]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/경남/경남_의령읍_결측처리.csv',index=False)

## 경남 - 이동면

In [373]:
## csv 파일들을 DataFrame으로 불러와서 concat
df = pd.read_csv(path+"경남_이동면_결측처리.csv",index_col=0)
    
data = df.reset_index(drop = True)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,0.6625,2.0250,48.125,53.125,6.0,-3.9,305.000,275.250,경남,이동면
1,2012-01-02,0.0,0.0,0.9125,-0.4375,50.000,45.625,6.0,-3.9,299.125,315.000,경남,이동면
2,2012-01-03,0.0,0.0,-1.0625,-0.3125,37.500,41.875,5.0,-3.9,315.000,303.750,경남,이동면
3,2012-01-04,0.0,0.0,-0.8125,0.7875,45.625,51.250,6.0,-4.9,309.375,298.125,경남,이동면
4,2012-01-05,0.0,0.0,-0.0750,0.8000,48.125,46.250,7.0,-3.9,291.750,214.125,경남,이동면


In [374]:
# 강수, 적설은 na 0으로 채움
rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)


# train / test 나눠서 interpolate
train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train=train.interpolate()
test=test.interpolate()

# 데이터 합치기
all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 
df3['date']=all_ymd

df_final = pd.concat([df3, rain_snow], axis=1)
df_final = df_final[['date','rainfall_24','snow_24','temp_24','temp_48','humid_24','humid_48','max_temp_24','min_temp_24','wind_24','wind_48','area','region']]
df_final

df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [375]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/경남/경남_이동면_결측처리.csv',index=False)

## 경남 - 장평동

In [376]:
## csv 파일들을 DataFrame으로 불러와서 concat
df = pd.read_csv(path+"경남_장평동_결측처리.csv",index_col=0)
    
data = df.reset_index(drop = True)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,1.1500,2.0250,47.500,54.375,7.0,-2.9,332.375,291.625,경남,장평동
1,2012-01-02,0.0,0.0,1.0375,-0.4375,50.000,45.625,6.0,-3.9,299.125,315.000,경남,장평동
2,2012-01-03,0.0,0.0,-1.1875,-0.5625,36.875,41.875,5.0,-4.9,315.000,315.000,경남,장평동
3,2012-01-04,0.0,0.0,-0.2000,0.7875,42.500,51.875,6.0,-3.9,309.250,298.125,경남,장평동
4,2012-01-05,0.0,0.0,0.0500,1.0250,48.750,48.750,7.0,-3.9,320.125,300.625,경남,장평동


In [377]:
# 강수, 적설은 na 0으로 채움
rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)


# train / test 나눠서 interpolate
train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train=train.interpolate()
test=test.interpolate()

# 데이터 합치기
all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 
df3['date']=all_ymd

df_final = pd.concat([df3, rain_snow], axis=1)
df_final = df_final[['date','rainfall_24','snow_24','temp_24','temp_48','humid_24','humid_48','max_temp_24','min_temp_24','wind_24','wind_48','area','region']]
df_final

df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [379]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/경남/경남_장평동_결측처리.csv',index=False)

## 경남 - 정량동

In [380]:
## csv 파일들을 DataFrame으로 불러와서 concat
df = pd.read_csv(path+"경남_정량동_결측처리.csv",index_col=0)
    
data = df.reset_index(drop = True)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,2.0250,2.5000,47.500,56.875,7.0,-1.9,45.000,320.625,경남,정량동
1,2012-01-02,0.0,0.0,1.4000,-0.0625,51.875,45.625,7.0,-2.9,299.125,315.000,경남,정량동
2,2012-01-03,0.0,0.0,-0.1875,0.6625,39.375,41.875,6.0,-3.9,315.000,360.000,경남,정량동
3,2012-01-04,0.0,0.0,0.2875,1.7750,46.875,50.625,7.0,-3.9,343.125,45.000,경남,정량동
4,2012-01-05,0.0,0.0,0.7875,1.5250,50.000,47.500,7.0,-3.9,313.625,260.750,경남,정량동


In [381]:
# 강수, 적설은 na 0으로 채움
rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)


# train / test 나눠서 interpolate
train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train=train.interpolate()
test=test.interpolate()

# 데이터 합치기
all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 
df3['date']=all_ymd

df_final = pd.concat([df3, rain_snow], axis=1)
df_final = df_final[['date','rainfall_24','snow_24','temp_24','temp_48','humid_24','humid_48','max_temp_24','min_temp_24','wind_24','wind_48','area','region']]
df_final

df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [382]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/경남/경남_정량동_결측처리.csv',index=False)

## 경남 - 평거동

In [383]:
## csv 파일들을 DataFrame으로 불러와서 concat
df = pd.read_csv(path+"경남_평거동_결측처리.csv",index_col=0)
    
data = df.reset_index(drop = True)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,-1.0625,0.0500,56.875,57.500,6.0,-7.9,170.125,235.750,경남,평거동
1,2012-01-02,0.0,0.0,-0.9375,-1.6875,57.500,43.750,6.0,-6.9,292.500,318.750,경남,평거동
2,2012-01-03,0.0,0.0,-2.3000,-1.5625,44.375,50.625,5.0,-7.9,315.375,317.125,경남,평거동
3,2012-01-04,0.0,0.0,-2.8125,-2.0625,50.625,53.750,7.0,-8.9,312.500,296.500,경남,평거동
4,2012-01-05,0.0,0.0,-1.5750,-1.0750,53.750,46.250,7.0,-8.9,162.750,141.375,경남,평거동


In [384]:
# 강수, 적설은 na 0으로 채움
rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)


# train / test 나눠서 interpolate
train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train=train.interpolate()
test=test.interpolate()

# 데이터 합치기
all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 
df3['date']=all_ymd

df_final = pd.concat([df3, rain_snow], axis=1)
df_final = df_final[['date','rainfall_24','snow_24','temp_24','temp_48','humid_24','humid_48','max_temp_24','min_temp_24','wind_24','wind_48','area','region']]
df_final

df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [385]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/경남/경남_평거동_결측처리.csv',index=False)

## 경남 - 함양읍

In [386]:
## csv 파일들을 DataFrame으로 불러와서 concat
df = pd.read_csv(path+"경남_함양읍_결측처리.csv",index_col=0)
    
data = df.reset_index(drop = True)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,-1.8250,-0.2375,58.500,64.750,5.8,-7.8,303.750,286.500,경남,함양읍
1,2012-01-02,0.0,0.0,-0.6125,-2.2750,61.625,47.875,4.8,-4.9,291.375,307.375,경남,함양읍
2,2012-01-03,1.0,1.0,-2.5000,-2.5750,52.500,56.000,3.8,-7.5,314.625,303.750,경남,함양읍
3,2012-01-04,0.0,0.0,-3.2500,-1.6375,53.500,60.250,4.9,-7.6,298.125,278.375,경남,함양읍
4,2012-01-05,0.0,0.0,-2.6125,-2.3000,59.000,48.125,5.9,-9.6,301.375,272.125,경남,함양읍


In [387]:
# 강수, 적설은 na 0으로 채움
rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)


# train / test 나눠서 interpolate
train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train=train.interpolate()
test=test.interpolate()

# 데이터 합치기
all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 
df3['date']=all_ymd

df_final = pd.concat([df3, rain_snow], axis=1)
df_final = df_final[['date','rainfall_24','snow_24','temp_24','temp_48','humid_24','humid_48','max_temp_24','min_temp_24','wind_24','wind_48','area','region']]
df_final

df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [388]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/경남/경남_함양읍_결측처리.csv',index=False)

## 경남 - 합천읍

In [389]:
## csv 파일들을 DataFrame으로 불러와서 concat
df = pd.read_csv(path+"경남_합천읍_결측처리.csv",index_col=0)
    
data = df.reset_index(drop = True)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,-1.2000,0.1750,57.500,65.000,7.0,-7.9,320.625,281.250,경남,합천읍
1,2012-01-02,0.0,0.0,-0.0750,-1.6875,60.625,45.625,6.0,-4.9,296.125,312.500,경남,합천읍
2,2012-01-03,0.0,0.0,-2.0625,-2.0625,51.250,55.625,5.0,-7.9,315.000,303.750,경남,합천읍
3,2012-01-04,0.0,0.0,-2.5625,-1.2000,51.875,60.000,6.0,-7.9,307.750,294.375,경남,합천읍
4,2012-01-05,0.0,0.0,-2.3125,-2.0750,58.750,48.750,7.0,-9.9,303.750,331.875,경남,합천읍


In [390]:
# 강수, 적설은 na 0으로 채움
rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)


# train / test 나눠서 interpolate
train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train=train.interpolate()
test=test.interpolate()

# 데이터 합치기
all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 
df3['date']=all_ymd

df_final = pd.concat([df3, rain_snow], axis=1)
df_final = df_final[['date','rainfall_24','snow_24','temp_24','temp_48','humid_24','humid_48','max_temp_24','min_temp_24','wind_24','wind_48','area','region']]
df_final

df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [391]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/경남/경남_합천읍_결측처리.csv',index=False)

# 광주

In [392]:
## csv 파일들을 DataFrame으로 불러와서 concat
df = pd.read_csv(path+"광주_운암1동_결측처리.csv",index_col=0)
    
data = df.reset_index(drop = True)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,5.0,5.0,-0.3250,0.425,67.500,79.375,5.0,-3.9,226.625,299.875,광주,운암1동
1,2012-01-02,0.0,0.0,0.0500,-1.675,69.375,65.000,4.0,-3.9,335.875,328.000,광주,운암1동
2,2012-01-03,5.0,5.0,-1.9125,-0.700,60.625,61.875,2.0,-3.9,327.000,352.500,광주,운암1동
3,2012-01-04,0.0,0.0,-1.8125,-0.075,66.250,65.000,5.0,-4.9,229.000,171.375,광주,운암1동
4,2012-01-05,0.0,0.0,-0.8125,-0.325,62.500,51.875,5.0,-4.9,163.000,156.625,광주,운암1동


In [393]:
# 강수, 적설은 na 0으로 채움
rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)


# train / test 나눠서 interpolate
train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train=train.interpolate()
test=test.interpolate()

# 데이터 합치기
all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 
df3['date']=all_ymd

df_final = pd.concat([df3, rain_snow], axis=1)
df_final = df_final[['date','rainfall_24','snow_24','temp_24','temp_48','humid_24','humid_48','max_temp_24','min_temp_24','wind_24','wind_48','area','region']]
df_final

df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [394]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/광주/광주_운암1동_결측처리.csv',index=False)

#대구

In [395]:
## csv 파일들을 DataFrame으로 불러와서 concat
df = pd.read_csv(path+"대구_효목1동_결측처리.csv",index_col=0)
    
data = df.reset_index(drop = True)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,-0.4625,1.1750,42.625,49.625,5.0,-5.0,312.250,292.875,대구,효목1동
1,2012-01-02,0.0,0.0,0.6250,-0.5125,48.250,45.500,5.0,-2.9,293.750,306.000,대구,효목1동
2,2012-01-03,0.0,0.0,-1.7000,-1.4250,45.875,40.500,3.0,-4.8,304.375,306.000,대구,효목1동
3,2012-01-04,0.0,0.0,-2.0875,-0.0875,39.750,39.625,3.9,-5.8,300.500,296.750,대구,효목1동
4,2012-01-05,0.0,0.0,-0.2750,-0.1000,35.375,34.500,5.0,-4.7,310.125,313.625,대구,효목1동


In [396]:
# 강수, 적설은 na 0으로 채움
rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)


# train / test 나눠서 interpolate
train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train=train.interpolate()
test=test.interpolate()

# 데이터 합치기
all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 
df3['date']=all_ymd

df_final = pd.concat([df3, rain_snow], axis=1)
df_final = df_final[['date','rainfall_24','snow_24','temp_24','temp_48','humid_24','humid_48','max_temp_24','min_temp_24','wind_24','wind_48','area','region']]
df_final

df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [397]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/대구/대구_효목1동_결측처리.csv',index=False)

# 대전

In [398]:
## csv 파일들을 DataFrame으로 불러와서 concat
df = pd.read_csv(path+"대전_온천2동_결측처리.csv",index_col=0)
    
data = df.reset_index(drop = True)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,-3.1750,-1.500,61.625,69.500,1.4,-9.2,0.9500,1.4625,대전,온천2동
1,2012-01-02,0.0,0.0,-2.1875,-3.425,65.125,52.000,1.5,-5.6,1.6125,2.4875,대전,온천2동
2,2012-01-03,0.0,0.0,-4.4000,-3.200,64.000,62.375,0.4,-8.7,2.6250,1.5875,대전,온천2동
3,2012-01-04,0.0,0.0,-4.3375,-3.000,63.750,61.625,2.7,-8.9,1.4750,1.3375,대전,온천2동
4,2012-01-05,0.0,0.0,-3.6125,-3.400,52.500,47.625,2.6,-9.2,1.4750,0.8875,대전,온천2동


In [399]:
# 강수, 적설은 na 0으로 채움
rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)


# train / test 나눠서 interpolate
train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train=train.interpolate()
test=test.interpolate()

# 데이터 합치기
all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 
df3['date']=all_ymd

df_final = pd.concat([df3, rain_snow], axis=1)
df_final = df_final[['date','rainfall_24','snow_24','temp_24','temp_48','humid_24','humid_48','max_temp_24','min_temp_24','wind_24','wind_48','area','region']]
df_final

df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [400]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/대전/대전_온천2동_결측처리.csv',index=False)

# 부산

In [401]:
## csv 파일들을 DataFrame으로 불러와서 concat
df = pd.read_csv(path+"부산_대청동_결측처리.csv",index_col=0)
    
data = df.reset_index(drop = True)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,0.0,0.0,2.4000,3.3750,39.375,48.125,7.0,-1.9,301.375,279.500,부산,대청동
1,2012-01-02,0.0,0.0,2.3750,1.2750,48.750,39.375,6.0,-1.9,282.625,293.875,부산,대청동
2,2012-01-03,0.0,0.0,0.4250,0.9000,38.125,36.875,6.0,-3.9,295.000,299.500,부산,대청동
3,2012-01-04,0.0,0.0,0.4125,1.7750,31.875,40.000,7.0,-3.9,300.375,290.750,부산,대청동
4,2012-01-05,0.0,0.0,0.6750,2.1625,31.875,31.875,8.0,-3.9,299.250,323.125,부산,대청동


In [402]:
# 강수, 적설은 na 0으로 채움
rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)


# train / test 나눠서 interpolate
train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train=train.interpolate()
test=test.interpolate()

# 데이터 합치기
all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 
df3['date']=all_ymd

df_final = pd.concat([df3, rain_snow], axis=1)
df_final = df_final[['date','rainfall_24','snow_24','temp_24','temp_48','humid_24','humid_48','max_temp_24','min_temp_24','wind_24','wind_48','area','region']]
df_final

df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [403]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/부산/부산_대청동_결측처리.csv',index=False)

# 서울

In [404]:
## csv 파일들을 DataFrame으로 불러와서 concat
df = pd.read_csv(path+"서울_교남동_결측처리.csv",index_col=0)
    
data = df.reset_index(drop = True)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

data.head()

,date,rainfall_24,snow_24,temp_24,temp_48,humid_24,humid_48,max_temp_24,min_temp_24,wind_24,wind_48,area,region
0,2012-01-01,1.0,1.0,-3.6625,-2.3000,45.625,54.375,0.0,-7.9,246.125,286.500,서울,교남동
1,2012-01-02,0.0,0.0,-3.6500,-5.9000,58.750,38.125,-0.9,-7.9,303.000,304.875,서울,교남동
2,2012-01-03,0.0,0.0,-6.5250,-4.9000,43.750,39.375,-0.9,-8.9,306.750,297.750,서울,교남동
3,2012-01-04,0.0,0.0,-4.9000,-2.1625,49.375,43.750,1.0,-9.9,266.375,233.250,서울,교남동
4,2012-01-05,0.0,0.0,-3.7875,-3.6625,39.375,40.625,1.0,-7.9,332.250,344.625,서울,교남동


In [405]:
# 강수, 적설은 na 0으로 채움
rain_snow = data[['rainfall_24','snow_24']]
data = data.drop(['rainfall_24','snow_24'],axis=1)

rain_snow = rain_snow.fillna(0)


# train / test 나눠서 interpolate
train = data.loc[data['date'].dt.year != 2016,:].drop('date', axis=1, inplace=False)
test = data.loc[data['date'].dt.year == 2016,:].drop('date', axis=1, inplace=False)

train=train.interpolate()
test=test.interpolate()

# 데이터 합치기
all_ymd = pd.date_range("2012-01-01", "2016-12-31")

df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df3 = pd.DataFrame(pd.concat([df1, df2], axis=0))
df3.columns = data.columns[1:] 
df3['date']=all_ymd

df_final = pd.concat([df3, rain_snow], axis=1)
df_final = df_final[['date','rainfall_24','snow_24','temp_24','temp_48','humid_24','humid_48','max_temp_24','min_temp_24','wind_24','wind_48','area','region']]
df_final

df_final.isna().sum()

date           0
rainfall_24    0
snow_24        0
temp_24        0
temp_48        0
humid_24       0
humid_48       0
max_temp_24    0
min_temp_24    0
wind_24        0
wind_48        0
area           0
region         0
dtype: int64

In [406]:
df_final.to_csv('/content/drive/MyDrive/날씨 빅콘/data/전처리데이터/예보데이터_new2/결측치처리_interpolate/서울/서울_교남동_결측처리.csv',index=False)